In [1]:
import numpy as np
import random
from itertools import permutations
from typing import List, Tuple, Optional, Callable
import time
import pulp
import copy
from CBS import cbs, load_map, get_random_free_position

In [2]:
class CapabilityProfile:
    def __init__(self,
                 robot_id: str,
                 mobility_type: str,
                #  max_speed: float,
                max_speed: int, # keep speed a constant integer of 1 for CBS purposes for now
                 payload_capacity: float,
                 reach: float,
                 battery_life: float,
                 size: Tuple[float, float, float],
                 environmental_resistance: List[str],
                 sensors: List[str],
                 sensor_range: float,
                 manipulators: List[str],
                 communication_protocols: List[str],
                 processing_power: float,
                 autonomy_level: str,
                 special_functions: List[str],
                 safety_features: List[str],
                 adaptability: bool,
                 location: Tuple[float, float, float],
                 preferred_tasks: List[str],
                 current_task: Optional["TaskDescription"],
                 remaining_distance: float,
                 time_on_task: float,
                 tasks_attempted: int,
                 tasks_successful: int,
                 current_path: list):
        self.robot_id = robot_id
        self.mobility_type = mobility_type
        self.max_speed = max_speed
        self.payload_capacity = payload_capacity
        self.reach = reach
        self.battery_life = battery_life
        self.size = size  # (length, width, height)
        self.environmental_resistance = environmental_resistance
        self.sensors = sensors
        self.sensor_range = sensor_range
        self.manipulators = manipulators
        self.communication_protocols = communication_protocols
        self.processing_power = processing_power
        self.autonomy_level = autonomy_level
        self.special_functions = special_functions
        self.safety_features = safety_features
        self.adaptability = adaptability
        self.location = location  # (x, y, z) coordinates
        self.preferred_tasks = preferred_tasks
        self.current_task = current_task  # The task the robot is currently assigned to
        self.remaining_distance = remaining_distance  # Distance left to the current task location
        self.time_on_task = time_on_task  # Time spent on the current task
        self.tasks_attempted = tasks_attempted
        self.tasks_successful = tasks_successful
        self.current_path = current_path

    def __repr__(self):
        return (f"CapabilityProfile(robot_id={self.robot_id}, mobility_type={self.mobility_type}, "
                f"max_speed={self.max_speed}, payload_capacity={self.payload_capacity}, "
                f"reach={self.reach}, battery_life={self.battery_life}, size={self.size}, "
                f"environmental_resistance={self.environmental_resistance}, sensors={self.sensors}, "
                f"sensor_range={self.sensor_range}, manipulators={self.manipulators}, communication_protocols={self.communication_protocols}, "
                f"processing_power={self.processing_power}, autonomy_level={self.autonomy_level}, "
                f"special_functions={self.special_functions}, safety_features={self.safety_features}, "
                f"adaptability={self.adaptability}, location={self.location}, preferred_tasks={self.preferred_tasks}, remaining_distance={self.remaining_distance}, time_on_task={self.time_on_task}, tasks_attempted={self.tasks_attempted}, tasks_successful={self.tasks_successful})")


class TaskDescription:
    def __init__(self,
                 task_id: str,
                 task_type: str,
                 objective: str,
                 priority_level: str,
                 reward: float,
                 difficulty: float,
                 location: Tuple[float, float, float],
                 navigation_constraints: Optional[List[str]],
                 required_capabilities: List[str],
                 time_window: Optional[Tuple[str, str]],
                 duration: float,
                 environmental_conditions: Optional[List[str]],
                 dependencies: Optional[List[str]],
                 tools_needed: Optional[List[str]],
                 communication_requirements: Optional[List[str]],
                 safety_protocols: Optional[List[str]],
                 performance_metrics: List[str],
                 success_criteria: str,
                 assigned_robot: Optional["CapabilityProfile"],
                 time_to_complete: float):
        self.task_id = task_id
        self.task_type = task_type
        self.objective = objective
        self.priority_level = priority_level
        self.reward = reward
        self.difficulty = difficulty
        self.location = location  # (x, y, z) coordinates
        self.navigation_constraints = navigation_constraints
        self.required_capabilities = required_capabilities
        self.time_window = time_window  # (start_time, end_time) or None
        self.duration = duration
        self.environmental_conditions = environmental_conditions
        self.dependencies = dependencies
        self.tools_needed = tools_needed
        self.communication_requirements = communication_requirements
        self.safety_protocols = safety_protocols
        self.performance_metrics = performance_metrics
        self.success_criteria = success_criteria
        self.assigned_robot = assigned_robot  # The robot currently assigned to this task
        self.time_to_complete = time_to_complete  # Time required to complete the task

    def __repr__(self):
        return (f"TaskDescription(task_id={self.task_id}, task_type={self.task_type}, "
                f"objective={self.objective}, priority_level={self.priority_level}, "
                f"location={self.location}, reward={self.reward}, difficulty={self.difficulty}, navigation_constraints={self.navigation_constraints}, "
                f"required_capabilities={self.required_capabilities}, time_window={self.time_window}, "
                f"duration={self.duration}, environmental_conditions={self.environmental_conditions}, "
                f"dependencies={self.dependencies}, tools_needed={self.tools_needed}, "
                f"communication_requirements={self.communication_requirements}, "
                f"safety_protocols={self.safety_protocols}, performance_metrics={self.performance_metrics}, "
                f"success_criteria={self.success_criteria}, time_to_complete={self.time_to_complete})")

In [3]:
def generate_random_robot_profile(robot_id: str, grid: List[List[int]], occupied_locations: set) -> CapabilityProfile:
    mobility_types = ["wheeled", "tracked", "legged", "aerial", "hovering", "climbing"]
    environmental_resistances = ["weatherproof", "waterproof", "dustproof", "heat-resistant", "cold-resistant", "shock-resistant"]
    sensors = ["camera", "microphone", "LiDAR", "GPS", "ultrasonic", "temperature sensor", "infrared", "proximity sensor", "magnetometer"]
    manipulators = ["gripper", "drill", "welding tool"]
    communication_protocols = ["Wi-Fi", "Bluetooth", "4G", "5G", "Radio"]
    special_functions = ["object recognition", "speech output", "facial recognition", "object tracking", "gesture recognition"]
    safety_features = ["emergency stop", "collision avoidance", "overheat protection", "fall detection", "obstacle detection", "speed reduction in crowded areas"]
    task_preferences = ["delivery", "inspection", "cleaning", "monitoring", "maintenance", "assembly", "surveying", "data collection", "assistance"]
    
    return CapabilityProfile(
        robot_id=robot_id,
        mobility_type=random.choice(mobility_types),
        # NOTE: MAKE CONSTANT SPEED
        # max_speed=random.uniform(0.5, 15.0),
        max_speed=1, # constant max speed for now, CBS purposes, macro time steps would increase complexity of time dimension slowing down CBS significantly
        payload_capacity=random.uniform(0.0, 50.0),
        reach=random.uniform(0.0, 10.0),
        battery_life=random.uniform(5.0, 500.0),
        size=(random.uniform(0.5, 5.0), random.uniform(0.5, 5.0), random.uniform(0.5, 5.0)),
        environmental_resistance=random.sample(environmental_resistances, k=random.randint(0, len(environmental_resistances))),
        sensors=random.sample(sensors, k=random.randint(1, len(sensors))),
        sensor_range=random.uniform(1.0, 50.0),
        manipulators=random.sample(manipulators, k=random.randint(0, len(manipulators))),
        communication_protocols=random.sample(communication_protocols, k=random.randint(1, len(communication_protocols))),
        processing_power=random.uniform(1.0, 10.0),
        autonomy_level=random.choice(["teleoperated", "semi-autonomous", "fully autonomous"]),
        special_functions=random.sample(special_functions, k=random.randint(0, len(special_functions))),
        safety_features=random.sample(safety_features, k=random.randint(0, len(safety_features))),
        adaptability=bool(random.getrandbits(1)),
        # NOTE: CHOOSE LOCATION FROM MAP WERE USING
        # location=(random.uniform(0.0, 50.0), random.uniform(0.0, 50.0), 0.0),
        location=get_random_free_position(grid, occupied_locations),
        preferred_tasks=random.sample(task_preferences, k=random.randint(0, len(task_preferences))),
        current_task = None,
        remaining_distance = 0.0,
        time_on_task = 0,
        tasks_attempted = 0,
        tasks_successful = 0,
        current_path = [] # holds the path from CBS solution
    )


def generate_random_task_description(task_id: str, grid: List[List[int]], occupied_locations: set) -> TaskDescription:
    task_types = ["delivery", "inspection", "cleaning", "monitoring", "maintenance", "assembly", "surveying", "data collection", "assistance"]
    priorities = ["low", "medium", "high", "urgent"]
    navigation_constraints = ["elevator", "stairs", "shelves", "no loud noises allowed", "narrow spaces", "low ceilings", "uneven floors", "low visibility", "slippery", "crowded", "loose debris", "no-fly zone", "windy", "dense obstructions", "smooth surfaces"]
    environmental_conditions = ["weatherproof", "waterproof", "dustproof", "heat-resistant", "cold-resistant", "shock-resistant"]
    performance_metrics = ["time taken", "accuracy", "energy consumption", "safety compliance", "completion rate"]
    requirements = [
        "payload capacity >= 5.0", 
        "payload capacity >= 1.0", 
        "payload capacity >= 10.0", 
        "payload capacity >= 20.0", 
        "reach >= 1.5", 
        "reach >= 3.0"
    ]
    tools = ["gripper", "camera", "microphone", "temperature sensor", "drill", "welding tool"]
    communication_types = ["Wi-Fi", "Bluetooth", "4G", "5G", "Radio"]
    safety_types = ["emergency stop", "collision avoidance", "overheat protection", "fall detection", "obstacle detection", "speed reduction in crowded areas"]
    duration_time = random.uniform(10.0, 100.0)
    
    return TaskDescription(
        task_id=task_id,
        task_type=random.choice(task_types),
        objective=f"Perform {random.choice(task_types)} task",
        priority_level=random.choice(priorities),
        reward=random.uniform(1, 10),
        difficulty=random.uniform(1, 10),
        # NOTE: CHOOSE LOCATION FROM MAP WERE USING
        # location=(random.uniform(0.0, 50.0), random.uniform(0.0, 50.0), 0.0),
        # NOTE: one assumption is that tasks can be created in occupied locations, doesnt matter if a robot is there, tasks just choose from all free area on map
        location = get_random_free_position(grid, occupied_locations),
        navigation_constraints=random.sample(navigation_constraints, k=random.randint(0, len(navigation_constraints))),
        required_capabilities=random.sample(requirements, k=random.randint(0, len(requirements))),
        time_window=(f"{random.randint(8, 17)}:00", f"{random.randint(18, 23)}:00"),
        duration=duration_time,
        environmental_conditions=random.sample(environmental_conditions, k=random.randint(0, len(environmental_conditions))),
        dependencies=None,
        tools_needed=random.sample(tools, k=random.randint(0, len(tools))),
        communication_requirements=random.sample(communication_types, k=random.randint(1, len(communication_types))),
        safety_protocols=random.sample(safety_types, k=random.randint(0, len(safety_types))),
        performance_metrics=random.sample(performance_metrics, k=random.randint(1, len(performance_metrics))),
        success_criteria="Task completed within time window",
        assigned_robot = None,
        time_to_complete = duration_time
    )

In [4]:
def navigation_suitability(robot_mobility_type: str, robot_size: Tuple[float, float, float], task_constraints: List[str]) -> float:
    """
    Evaluates the suitability of a robot for navigating a task environment based on mobility type, size, and navigation constraints.
    
    Parameters:
    - robot_mobility_type: The mobility type of the robot (e.g., "wheeled", "tracked", "legged", "aerial", "hovering", "climbing").
    - robot_size: A tuple representing the robot's dimensions (length, width, height).
    - task_constraints: A list of navigation constraints for the task environment.
    
    Returns:
    - A float score representing the suitability for navigation. Returns 0 if there is a critical mismatch that prevents navigation.
    """

    # Initialize the score
    score = 0.0

    # Define size thresholds for narrow spaces, low ceilings, etc.
    narrow_space_threshold = 1.0  # Width limit for narrow spaces
    low_ceiling_threshold = 1.5   # Height limit for low ceilings

    # Handle each task constraint based on mobility type
    for constraint in task_constraints:
        # Constraint: Elevator access
        if constraint == "elevator":
            if robot_mobility_type in ["wheeled", "tracked", "legged"]:
                score += 1.0

        # Constraint: Stairs
        elif constraint == "stairs":
            if robot_mobility_type in ["legged", "aerial", "climbing"]:
                score += 1.0

        # Constraint: Shelves
        elif constraint == "shelves":
            if robot_size[2] < low_ceiling_threshold or robot_mobility_type in ["aerial", "climbing", "hovering"]:
                score += 1.0  # Only smaller robots can access shelves effectively

        # Constraint: No loud noises allowed
        elif constraint == "no loud noises allowed":
            if robot_mobility_type in ["legged", "hovering"]:
                score += 1.0  # Quieter mobility types

        # Constraint: Narrow spaces
        elif constraint == "narrow spaces":
            if robot_size[1] <= narrow_space_threshold:
                score += 1.0
            else:
                return 0.0  # Larger robots cannot pass through narrow spaces

        # Constraint: Low ceilings
        elif constraint == "low ceilings":
            if robot_size[2] <= low_ceiling_threshold:
                score += 1.0
            else:
                return 0.0  # Tall robots cannot navigate in areas with low ceilings

        # Constraint: Uneven floors
        elif constraint == "uneven floors":
            if robot_mobility_type in ["tracked", "legged"]:
                score += 1.0  # These types handle uneven floors well

        # Constraint: Low visibility
        elif constraint == "low visibility":
            if robot_mobility_type in ["wheeled", "tracked", "legged"]:
                score += 1.0  # Infrared or LiDAR-equipped robots are suitable

        # Constraint: Slippery surfaces
        elif constraint == "slippery":
            if robot_mobility_type in ["tracked", "hovering", "aerial"]:
                score += 1.0  # Hovering and tracked types handle slippery surfaces better
            elif robot_mobility_type in ["wheeled", "legged"]:
                return 0.0  # Wheeled and legged robots are unsuitable on slippery floors

        # Constraint: Crowded environments
        elif constraint == "crowded":
            if robot_size[0] <= 1.0 and robot_size[1] <= 1.0:
                score += 1.0  # Smaller robots are more suitable in crowded environments
            else:
                score += 0.5  # Larger robots get a lower score

        # Constraint: Loose debris
        elif constraint == "loose debris":
            if robot_mobility_type in ["aerial", "hovering"]:
                score += 1.0  # Aerial and hovering robots handle debris better
            elif robot_mobility_type == "legged":
                return 0.0  # Legged robots are unsuitable

        # Constraint: No-fly zone
        elif constraint == "no-fly zone":
            if robot_mobility_type == "aerial":
                return 0.0  # Aerial robots cannot navigate in no-fly zones
            else:
                score += 0.5  # Other mobility types are unaffected

        # Constraint: Windy conditions
        elif constraint == "windy":
            if robot_mobility_type == "aerial":
                return 0.0  # Aerial robots struggle in windy conditions
            else:
                score += 0.5  # All other types are more stable in wind

        # Constraint: Dense obstructions (e.g., tree branches, hanging cables)
        elif constraint == "dense obstructions":
            if robot_mobility_type in ["aerial", "legged"]:
                return 0.0  # Aerial and legged robots are unsuitable in dense obstruction areas
            else:
                score += 0.5  # Other types may navigate dense areas on the ground

        # Constraint: Smooth floors
        elif constraint == "smooth surfaces":
            if robot_mobility_type == "climbing":
                return 0.0  # Climbing robots are less suited for smooth surfaces

    # Final suitability score (0 if any constraint returns 0)
    return score if score > 0 else 0.0


def evaluate_suitability_loose(robot: CapabilityProfile, task: TaskDescription) -> float:
    """
    Evaluates the suitability of a robot for a given task.
    A higher score indicates better suitability.
    
    Parameters:
    - robot: The CapabilityProfile of the robot.
    - task: The TaskDescription of the task.
    
    Returns:
    - A float score representing the suitability of the robot for the task. A score of 0 indicates the robot cannot perform the task.
    """
    score = 0.0
    
#     print(task.required_capabilities, robot.payload_capacity)
    # Check if robot meets the minimum requirements
    if any(req for req in task.required_capabilities if "payload capacity" in req and robot.payload_capacity < float(req.split(">= ")[-1])):
        score += 0.0  # Suitability is zero if the robot doesn't meet minimum requirements
    else:
        score += 1.0  # Add score if payload meets or exceeds requirements
    
#     print(task.tools_needed, robot.sensors+robot.manipulators)
    # Check if the robot has the necessary tools for the task
    if task.tools_needed and not all(item in robot.sensors+robot.manipulators for item in task.tools_needed):
        score += 0.0  # Suitability is zero if the robot lacks necessary tools
    else:
        score += 1.0  # Add score if robot has necessary tools
    
#     print(task.communication_requirements, robot.communication_protocols)
    # Check if the robot can communicate as required by the task
    if task.communication_requirements and not all(protocol in robot.communication_protocols for protocol in task.communication_requirements):
        score += 0.0  # Suitability is zero if the robot lacks required communication protocols
    else:
        score += 1.0  # Add score if robot has communication requirements
    
#     print(task.safety_protocols, robot.safety_features)
    # Check if the robot can safely perform the task
    if task.safety_protocols and not all(safety in robot.safety_features for safety in task.safety_protocols):
        score += 0.0  # Suitability is zero if the robot lacks required safety features
    else:
        score += 1.0  # Add score if robot meets safety requirements
    
#     print(task.environmental_conditions, robot.environmental_resistance)
    # Environmental compatibility: Can the robot operate in the task’s conditions?
    if task.environmental_conditions and not all(condition in robot.environmental_resistance for condition in task.environmental_conditions):
        score += 0.0  # Suitability is zero if the robot can't operate in required environmental conditions
    else:
        score += 1.0  # Add score if robot has required environmental resistances
    
#     print(task.required_capabilities, robot.reach)
    # Check if the robot meets reach requirements
    if any(req for req in task.required_capabilities if "reach" in req and robot.reach < float(req.split(">= ")[-1])):
        score += 0.0  # Suitability is zero if the robot cannot reach the task area as required
    else:
        score += 1.0  # Add score if reach meets or exceeds requirements
    
#     print(task.navigation_constraints, robot.mobility_type, robot.size)
    # Check navigation constraints based on mobility type and robot size
    navigation_match = navigation_suitability(robot.mobility_type, robot.size, task.navigation_constraints)
    if navigation_match == 0:
        score += 0.0
    else:
        score += navigation_match

    # NOTE: CHANGE TO WORK WITH COORDINATES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    # distance_to_task = ((robot.location[0] - task.location[0]) ** 2 + (robot.location[1] - task.location[1]) ** 2) ** 0.5
    distance_to_task = len(robot.current_path) - 1
#     print(robot.sensor_range)
    # Check sensor capabilities for the task
    if robot.sensor_range >= distance_to_task:
        score += 1.0
    elif robot.sensor_range >= distance_to_task/2:
        score += 0.5

    # Battery and distance check: Ensure the robot has sufficient battery to reach and complete the task
#     print(robot.max_speed, robot.battery_life, task.duration, distance_to_task)
    if ((distance_to_task / robot.max_speed)+task.duration) > robot.battery_life:
        score += 0.0  # Suitability is zero if the robot can't complete the task due to distance, speed, or battery limitations

    # Add to score based on proximity (closer robots get higher scores)
    if distance_to_task < 20.0:
        score += 1.0
    elif distance_to_task < 50.0:
        score += 0.5

#     print(task.priority_level, robot.autonomy_level)
    # Check if the robot's autonomy level matches the task's priority level
    if task.priority_level in ["high", "urgent"] and robot.autonomy_level in ["fully autonomous", "teleoperated"]:
        score += 1.0
    elif task.priority_level in ["medium", "low"] and robot.autonomy_level in ["semi-autonomous", "fully autonomous"]:
        score += 0.5

#     print(robot.battery_life, task.duration)
    # Evaluate battery life for task duration
    if robot.battery_life >= 2*((distance_to_task / robot.max_speed)+task.duration):
        score += 1.0
    else:
        score += 0.5

#     print(task.task_type, robot.special_functions)
    task_function_mapping = {
        "delivery": ["object recognition", "speech output", "facial recognition"],
        "inspection": ["object recognition", "object tracking", "gesture recognition"],
        "cleaning": ["object recognition"],
        "monitoring": ["speech output", "object tracking", "facial recognition"],
        "maintenance": ["object recognition", "path planning"],
        "assembly": ["object recognition"],
        "surveying": ["speech output", "facial recognition", "object recognition", "object tracking"],
        "data collection": ["object recognition", "object tracking", "facial recognition", "gesture recognition"],
        "assistance": ["speech output", "facial recognition", "gesture recognition"]
    }

    # Get the relevant functions for this task type
    required_functions = task_function_mapping[task.task_type]

    # Calculate the score based on matches between robot's functions and required functions
    for function in robot.special_functions:
        if function in required_functions:
            score += 1.0  # Increase score for each match
    
#     # Dependencies
#     if task.dependencies:
#         # Assume dependencies are represented as tasks that must be completed first
#         score += 0.5 if all(dep in completed_tasks for dep in task.dependencies) else 0.0
    
#     print(task.difficulty, robot.processing_power)
    # Processing power: Certain tasks may benefit from higher processing power if they are computationally demanding
    if task.difficulty > 7 and robot.processing_power >= 5.0:  # Difficulty > 7 indicates a complex task
        score += 1.0
    elif task.difficulty > 4 and robot.processing_power >= 3.0:
        score += 1.0
    elif task.difficulty > 2 and robot.processing_power >= 1.5:
        score += 0.5

#     print(robot.adaptability)
    # Consider robot's adaptability to changing conditions
    if robot.adaptability:
        score += 0.5
    
#     print(task.task_type, robot.preferred_tasks)
    # Preference matching
    if task.task_type in robot.preferred_tasks:
        score += 1.0

    # Score based on priority, reward, and difficulty
    priority_multiplier = {"low": 0.5, "medium": 1.0, "high": 1.5, "urgent": 2.0}[task.priority_level]
    reward_to_difficulty_ratio = task.reward / task.difficulty
#     print(task.priority_level, task.reward, task.difficulty, priority_multiplier, reward_to_difficulty_ratio)
    score += priority_multiplier * reward_to_difficulty_ratio

    # Return the final suitability score
#     print(score)
    return score

def evaluate_suitability_strict(robot: CapabilityProfile, task: TaskDescription) -> float:
    """
    Evaluates the suitability of a robot for a given task.
    A higher score indicates better suitability.
    
    Parameters:
    - robot: The CapabilityProfile of the robot.
    - task: The TaskDescription of the task.
    
    Returns:
    - A float score representing the suitability of the robot for the task. A score of 0 indicates the robot cannot perform the task.
    """
    score = 0.0
    
#     print(task.required_capabilities, robot.payload_capacity)
    # Check if robot meets the minimum requirements
    if any(req for req in task.required_capabilities if "payload capacity" in req and robot.payload_capacity < float(req.split(">= ")[-1])):
        return 0.0  # Suitability is zero if the robot doesn't meet minimum requirements
    else:
        score += 1.0  # Add score if payload meets or exceeds requirements
    
#     print(task.tools_needed, robot.sensors+robot.manipulators)
    # Check if the robot has the necessary tools for the task
    if task.tools_needed and not all(item in robot.sensors+robot.manipulators for item in task.tools_needed):
        return 0.0  # Suitability is zero if the robot lacks necessary tools
    else:
        score += 1.0  # Add score if robot has necessary tools
    
#     print(task.communication_requirements, robot.communication_protocols)
    # Check if the robot can communicate as required by the task
    if task.communication_requirements and not all(protocol in robot.communication_protocols for protocol in task.communication_requirements):
        return 0.0  # Suitability is zero if the robot lacks required communication protocols
    else:
        score += 1.0  # Add score if robot has communication requirements
    
#     print(task.safety_protocols, robot.safety_features)
    # Check if the robot can safely perform the task
    if task.safety_protocols and not all(safety in robot.safety_features for safety in task.safety_protocols):
        return 0.0  # Suitability is zero if the robot lacks required safety features
    else:
        score += 1.0  # Add score if robot meets safety requirements
    
#     print(task.environmental_conditions, robot.environmental_resistance)
    # Environmental compatibility: Can the robot operate in the task’s conditions?
    if task.environmental_conditions and not all(condition in robot.environmental_resistance for condition in task.environmental_conditions):
        return 0.0  # Suitability is zero if the robot can't operate in required environmental conditions
    else:
        score += 1.0  # Add score if robot has required environmental resistances
    
#     print(task.required_capabilities, robot.reach)
    # Check if the robot meets reach requirements
    if any(req for req in task.required_capabilities if "reach" in req and robot.reach < float(req.split(">= ")[-1])):
        return 0.0  # Suitability is zero if the robot cannot reach the task area as required
    else:
        score += 1.0  # Add score if reach meets or exceeds requirements
    
#     print(task.navigation_constraints, robot.mobility_type, robot.size)
    # Check navigation constraints based on mobility type and robot size
    navigation_match = navigation_suitability(robot.mobility_type, robot.size, task.navigation_constraints)
    if navigation_match == 0:
        return 0.0
    else:
        score += navigation_match

    # NOTE: CHANGE TO WORK WITH COORDINATES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    # distance_to_task = ((robot.location[0] - task.location[0]) ** 2 + (robot.location[1] - task.location[1]) ** 2) ** 0.5
    distance_to_task = len(robot.current_path) - 1
#     print(robot.sensor_range)
    # Check sensor capabilities for the task
    if robot.sensor_range >= distance_to_task:
        score += 1.0
    elif robot.sensor_range >= distance_to_task/2:
        score += 0.5

    # Battery and distance check: Ensure the robot has sufficient battery to reach and complete the task
#     print(robot.max_speed, robot.battery_life, task.duration, distance_to_task)
    if ((distance_to_task / robot.max_speed)+task.duration) > robot.battery_life:
        return 0.0  # Suitability is zero if the robot can't complete the task due to distance, speed, or battery limitations

    # Add to score based on proximity (closer robots get higher scores)
    if distance_to_task < 20.0:
        score += 1.0
    elif distance_to_task < 50.0:
        score += 0.5

#     print(task.priority_level, robot.autonomy_level)
    # Check if the robot's autonomy level matches the task's priority level
    if task.priority_level in ["high", "urgent"] and robot.autonomy_level in ["fully autonomous", "teleoperated"]:
        score += 1.0
    elif task.priority_level in ["medium", "low"] and robot.autonomy_level in ["semi-autonomous", "fully autonomous"]:
        score += 0.5

#     print(robot.battery_life, task.duration)
    # Evaluate battery life for task duration
    if robot.battery_life >= 2*((distance_to_task / robot.max_speed)+task.duration):
        score += 1.0
    else:
        score += 0.5

#     print(task.task_type, robot.special_functions)
    task_function_mapping = {
        "delivery": ["object recognition", "speech output", "facial recognition"],
        "inspection": ["object recognition", "object tracking", "gesture recognition"],
        "cleaning": ["object recognition"],
        "monitoring": ["speech output", "object tracking", "facial recognition"],
        "maintenance": ["object recognition", "path planning"],
        "assembly": ["object recognition"],
        "surveying": ["speech output", "facial recognition", "object recognition", "object tracking"],
        "data collection": ["object recognition", "object tracking", "facial recognition", "gesture recognition"],
        "assistance": ["speech output", "facial recognition", "gesture recognition"]
    }

    # Get the relevant functions for this task type
    required_functions = task_function_mapping[task.task_type]

    # Calculate the score based on matches between robot's functions and required functions
    for function in robot.special_functions:
        if function in required_functions:
            score += 1.0  # Increase score for each match
    
#     # Dependencies
#     if task.dependencies:
#         # Assume dependencies are represented as tasks that must be completed first
#         score += 0.5 if all(dep in completed_tasks for dep in task.dependencies) else 0.0
    
#     print(task.difficulty, robot.processing_power)
    # Processing power: Certain tasks may benefit from higher processing power if they are computationally demanding
    if task.difficulty > 7 and robot.processing_power >= 5.0:  # Difficulty > 7 indicates a complex task
        score += 1.0
    elif task.difficulty > 4 and robot.processing_power >= 3.0:
        score += 1.0
    elif task.difficulty > 2 and robot.processing_power >= 1.5:
        score += 0.5

#     print(robot.adaptability)
    # Consider robot's adaptability to changing conditions
    if robot.adaptability:
        score += 0.5
    
#     print(task.task_type, robot.preferred_tasks)
    # Preference matching
    if task.task_type in robot.preferred_tasks:
        score += 1.0

    # Score based on priority, reward, and difficulty
    priority_multiplier = {"low": 0.5, "medium": 1.0, "high": 1.5, "urgent": 2.0}[task.priority_level]
    reward_to_difficulty_ratio = task.reward / task.difficulty
#     print(task.priority_level, task.reward, task.difficulty, priority_multiplier, reward_to_difficulty_ratio)
    score += priority_multiplier * reward_to_difficulty_ratio

    # Return the final suitability score
#     print(score)
    return score

def evaluate_suitability_distance(robot: CapabilityProfile, task: TaskDescription) -> float:
    """
    Evaluates the suitability of a robot for a given task.
    A higher score indicates better suitability.
    
    Parameters:
    - robot: The CapabilityProfile of the robot.
    - task: The TaskDescription of the task.
    
    Returns:
    - A float score representing the suitability of the robot for the task. A score of 0 indicates the robot cannot perform the task.
    """
    score = 0.0
    
#     print(task.required_capabilities, robot.payload_capacity)
    # Check if robot meets the minimum requirements
    if any(req for req in task.required_capabilities if "payload capacity" in req and robot.payload_capacity < float(req.split(">= ")[-1])):
        score += 0.0  # Suitability is zero if the robot doesn't meet minimum requirements
    else:
        score += 1.0  # Add score if payload meets or exceeds requirements
    
#     print(task.tools_needed, robot.sensors+robot.manipulators)
    # Check if the robot has the necessary tools for the task
    if task.tools_needed and not all(item in robot.sensors+robot.manipulators for item in task.tools_needed):
        score += 0.0  # Suitability is zero if the robot lacks necessary tools
    else:
        score += 1.0  # Add score if robot has necessary tools
    
#     print(task.communication_requirements, robot.communication_protocols)
    # Check if the robot can communicate as required by the task
    if task.communication_requirements and not all(protocol in robot.communication_protocols for protocol in task.communication_requirements):
        score += 0.0  # Suitability is zero if the robot lacks required communication protocols
    else:
        score += 1.0  # Add score if robot has communication requirements
    
#     print(task.safety_protocols, robot.safety_features)
    # Check if the robot can safely perform the task
    if task.safety_protocols and not all(safety in robot.safety_features for safety in task.safety_protocols):
        score += 0.0  # Suitability is zero if the robot lacks required safety features
    else:
        score += 1.0  # Add score if robot meets safety requirements
    
#     print(task.environmental_conditions, robot.environmental_resistance)
    # Environmental compatibility: Can the robot operate in the task’s conditions?
    if task.environmental_conditions and not all(condition in robot.environmental_resistance for condition in task.environmental_conditions):
        score += 0.0  # Suitability is zero if the robot can't operate in required environmental conditions
    else:
        score += 1.0  # Add score if robot has required environmental resistances
    
#     print(task.required_capabilities, robot.reach)
    # Check if the robot meets reach requirements
    if any(req for req in task.required_capabilities if "reach" in req and robot.reach < float(req.split(">= ")[-1])):
        score += 0.0  # Suitability is zero if the robot cannot reach the task area as required
    else:
        score += 1.0  # Add score if reach meets or exceeds requirements
    
#     print(task.navigation_constraints, robot.mobility_type, robot.size)
    # Check navigation constraints based on mobility type and robot size
    navigation_match = navigation_suitability(robot.mobility_type, robot.size, task.navigation_constraints)
    if navigation_match == 0:
        score += 0.0
    else:
        score += navigation_match

    # NOTE: CHANGE TO WORK WITH COORDINATES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    # distance_to_task = ((robot.location[0] - task.location[0]) ** 2 + (robot.location[1] - task.location[1]) ** 2) ** 0.5
    distance_to_task = len(robot.current_path) - 1
#     print(robot.sensor_range)
    # Check sensor capabilities for the task
    if robot.sensor_range >= distance_to_task:
        score += 1.0
    elif robot.sensor_range >= distance_to_task/2:
        score += 0.5

    # Battery and distance check: Ensure the robot has sufficient battery to reach and complete the task
#     print(robot.max_speed, robot.battery_life, task.duration, distance_to_task)
    if ((distance_to_task / robot.max_speed)+task.duration) > robot.battery_life:
        score += 0.0  # Suitability is zero if the robot can't complete the task due to distance, speed, or battery limitations

    # Add to score based on proximity (closer robots get higher scores)
#     if distance_to_task < 20.0:
#         score += 1.0
#     elif distance_to_task < 50.0:
#         score += 0.5

#     print(task.priority_level, robot.autonomy_level)
    # Check if the robot's autonomy level matches the task's priority level
    if task.priority_level in ["high", "urgent"] and robot.autonomy_level in ["fully autonomous", "teleoperated"]:
        score += 1.0
    elif task.priority_level in ["medium", "low"] and robot.autonomy_level in ["semi-autonomous", "fully autonomous"]:
        score += 0.5

#     print(robot.battery_life, task.duration)
    # Evaluate battery life for task duration
    if robot.battery_life >= 2*((distance_to_task / robot.max_speed)+task.duration):
        score += 1.0
    else:
        score += 0.5

#     print(task.task_type, robot.special_functions)
    task_function_mapping = {
        "delivery": ["object recognition", "speech output", "facial recognition"],
        "inspection": ["object recognition", "object tracking", "gesture recognition"],
        "cleaning": ["object recognition"],
        "monitoring": ["speech output", "object tracking", "facial recognition"],
        "maintenance": ["object recognition", "path planning"],
        "assembly": ["object recognition"],
        "surveying": ["speech output", "facial recognition", "object recognition", "object tracking"],
        "data collection": ["object recognition", "object tracking", "facial recognition", "gesture recognition"],
        "assistance": ["speech output", "facial recognition", "gesture recognition"]
    }

    # Get the relevant functions for this task type
    required_functions = task_function_mapping[task.task_type]

    # Calculate the score based on matches between robot's functions and required functions
    for function in robot.special_functions:
        if function in required_functions:
            score += 1.0  # Increase score for each match
    
#     # Dependencies
#     if task.dependencies:
#         # Assume dependencies are represented as tasks that must be completed first
#         score += 0.5 if all(dep in completed_tasks for dep in task.dependencies) else 0.0
    
#     print(task.difficulty, robot.processing_power)
    # Processing power: Certain tasks may benefit from higher processing power if they are computationally demanding
    if task.difficulty > 7 and robot.processing_power >= 5.0:  # Difficulty > 7 indicates a complex task
        score += 1.0
    elif task.difficulty > 4 and robot.processing_power >= 3.0:
        score += 1.0
    elif task.difficulty > 2 and robot.processing_power >= 1.5:
        score += 0.5

#     print(robot.adaptability)
    # Consider robot's adaptability to changing conditions
    if robot.adaptability:
        score += 0.5
    
#     print(task.task_type, robot.preferred_tasks)
    # Preference matching
    if task.task_type in robot.preferred_tasks:
        score += 1.0

    # Score based on priority, reward, and difficulty
    priority_multiplier = {"low": 0.5, "medium": 1.0, "high": 1.5, "urgent": 2.0}[task.priority_level]
    reward_to_difficulty_ratio = task.reward / task.difficulty
#     print(task.priority_level, task.reward, task.difficulty, priority_multiplier, reward_to_difficulty_ratio)
    score += priority_multiplier * reward_to_difficulty_ratio

    # Weight score by distance to task
    score = score / distance_to_task
    
    # Return the final suitability score
#     print(score)
    return score

def evaluate_suitability_priority(robot: CapabilityProfile, task: TaskDescription) -> float:
    """
    Evaluates the suitability of a robot for a given task.
    A higher score indicates better suitability.
    
    Parameters:
    - robot: The CapabilityProfile of the robot.
    - task: The TaskDescription of the task.
    
    Returns:
    - A float score representing the suitability of the robot for the task. A score of 0 indicates the robot cannot perform the task.
    """
    score = 0.0
    
#     print(task.required_capabilities, robot.payload_capacity)
    # Check if robot meets the minimum requirements
    if any(req for req in task.required_capabilities if "payload capacity" in req and robot.payload_capacity < float(req.split(">= ")[-1])):
        score += 0.0  # Suitability is zero if the robot doesn't meet minimum requirements
    else:
        score += 1.0  # Add score if payload meets or exceeds requirements
    
#     print(task.tools_needed, robot.sensors+robot.manipulators)
    # Check if the robot has the necessary tools for the task
    if task.tools_needed and not all(item in robot.sensors+robot.manipulators for item in task.tools_needed):
        score += 0.0  # Suitability is zero if the robot lacks necessary tools
    else:
        score += 1.0  # Add score if robot has necessary tools
    
#     print(task.communication_requirements, robot.communication_protocols)
    # Check if the robot can communicate as required by the task
    if task.communication_requirements and not all(protocol in robot.communication_protocols for protocol in task.communication_requirements):
        score += 0.0  # Suitability is zero if the robot lacks required communication protocols
    else:
        score += 1.0  # Add score if robot has communication requirements
    
#     print(task.safety_protocols, robot.safety_features)
    # Check if the robot can safely perform the task
    if task.safety_protocols and not all(safety in robot.safety_features for safety in task.safety_protocols):
        score += 0.0  # Suitability is zero if the robot lacks required safety features
    else:
        score += 1.0  # Add score if robot meets safety requirements
    
#     print(task.environmental_conditions, robot.environmental_resistance)
    # Environmental compatibility: Can the robot operate in the task’s conditions?
    if task.environmental_conditions and not all(condition in robot.environmental_resistance for condition in task.environmental_conditions):
        score += 0.0  # Suitability is zero if the robot can't operate in required environmental conditions
    else:
        score += 1.0  # Add score if robot has required environmental resistances
    
#     print(task.required_capabilities, robot.reach)
    # Check if the robot meets reach requirements
    if any(req for req in task.required_capabilities if "reach" in req and robot.reach < float(req.split(">= ")[-1])):
        score += 0.0  # Suitability is zero if the robot cannot reach the task area as required
    else:
        score += 1.0  # Add score if reach meets or exceeds requirements
    
#     print(task.navigation_constraints, robot.mobility_type, robot.size)
    # Check navigation constraints based on mobility type and robot size
    navigation_match = navigation_suitability(robot.mobility_type, robot.size, task.navigation_constraints)
    if navigation_match == 0:
        score += 0.0
    else:
        score += navigation_match

    # NOTE: CHANGE TO WORK WITH COORDINATES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    # distance_to_task = ((robot.location[0] - task.location[0]) ** 2 + (robot.location[1] - task.location[1]) ** 2) ** 0.5
    distance_to_task = len(robot.current_path) - 1
#     print(robot.sensor_range)
    # Check sensor capabilities for the task
    if robot.sensor_range >= distance_to_task:
        score += 1.0
    elif robot.sensor_range >= distance_to_task/2:
        score += 0.5

    # Battery and distance check: Ensure the robot has sufficient battery to reach and complete the task
#     print(robot.max_speed, robot.battery_life, task.duration, distance_to_task)
    if ((distance_to_task / robot.max_speed)+task.duration) > robot.battery_life:
        score += 0.0  # Suitability is zero if the robot can't complete the task due to distance, speed, or battery limitations

    # Add to score based on proximity (closer robots get higher scores)
    if distance_to_task < 20.0:
        score += 1.0
    elif distance_to_task < 50.0:
        score += 0.5

#     print(task.priority_level, robot.autonomy_level)
    # Check if the robot's autonomy level matches the task's priority level
    if task.priority_level in ["high", "urgent"] and robot.autonomy_level in ["fully autonomous", "teleoperated"]:
        score += 1.0
    elif task.priority_level in ["medium", "low"] and robot.autonomy_level in ["semi-autonomous", "fully autonomous"]:
        score += 0.5

#     print(robot.battery_life, task.duration)
    # Evaluate battery life for task duration
    if robot.battery_life >= 2*((distance_to_task / robot.max_speed)+task.duration):
        score += 1.0
    else:
        score += 0.5

#     print(task.task_type, robot.special_functions)
    task_function_mapping = {
        "delivery": ["object recognition", "speech output", "facial recognition"],
        "inspection": ["object recognition", "object tracking", "gesture recognition"],
        "cleaning": ["object recognition"],
        "monitoring": ["speech output", "object tracking", "facial recognition"],
        "maintenance": ["object recognition", "path planning"],
        "assembly": ["object recognition"],
        "surveying": ["speech output", "facial recognition", "object recognition", "object tracking"],
        "data collection": ["object recognition", "object tracking", "facial recognition", "gesture recognition"],
        "assistance": ["speech output", "facial recognition", "gesture recognition"]
    }

    # Get the relevant functions for this task type
    required_functions = task_function_mapping[task.task_type]

    # Calculate the score based on matches between robot's functions and required functions
    for function in robot.special_functions:
        if function in required_functions:
            score += 1.0  # Increase score for each match
    
#     # Dependencies
#     if task.dependencies:
#         # Assume dependencies are represented as tasks that must be completed first
#         score += 0.5 if all(dep in completed_tasks for dep in task.dependencies) else 0.0
    
#     print(task.difficulty, robot.processing_power)
    # Processing power: Certain tasks may benefit from higher processing power if they are computationally demanding
    if task.difficulty > 7 and robot.processing_power >= 5.0:  # Difficulty > 7 indicates a complex task
        score += 1.0
    elif task.difficulty > 4 and robot.processing_power >= 3.0:
        score += 1.0
    elif task.difficulty > 2 and robot.processing_power >= 1.5:
        score += 0.5

#     print(robot.adaptability)
    # Consider robot's adaptability to changing conditions
    if robot.adaptability:
        score += 0.5
    
#     print(task.task_type, robot.preferred_tasks)
    # Preference matching
    if task.task_type in robot.preferred_tasks:
        score += 1.0

    # Score based on reward and difficulty
    priority_multiplier = {"low": 0.5, "medium": 1.0, "high": 1.5, "urgent": 2.0}[task.priority_level]
    reward_to_difficulty_ratio = task.reward / task.difficulty
#     print(task.priority_level, task.reward, task.difficulty, priority_multiplier, reward_to_difficulty_ratio)
    score += reward_to_difficulty_ratio

    # Weight based on priority
    score = score * priority_multiplier
    
    # Return the final suitability score
#     print(score)
    return score

In [5]:
def evaluate_suitability_with_llm(robot: CapabilityProfile, task: TaskDescription) -> str:
    """
    Uses a large language model (LLM) to evaluate the suitability of a robot for a given task.

    Parameters:
    - robot: The CapabilityProfile of the robot.
    - task: The TaskDescription of the task.

    Returns:
    - A float score (0 to 10) representing the suitability of the robot for the task.
    """
    # Construct a detailed prompt describing the scenario
    prompt = f"""
    You are evaluating the suitability of a robot for a specific task in a robotic system.

    Here are the details of the robot:
    - Mobility Type: {robot.mobility_type}
    - Maximum Speed: {robot.max_speed} units/sec
    - Payload Capacity: {robot.payload_capacity} kg
    - Reach: {robot.reach} meters
    - Battery Life: {robot.battery_life} hours
    - Size: {robot.size} (length, width, height in meters)
    - Environmental Resistance: {', '.join(robot.environmental_resistance)}
    - Sensors: {', '.join(robot.sensors)}
    - Sensor Range: {robot.sensor_range} meters
    - Manipulators: {', '.join(robot.manipulators)}
    - Communication Protocols: {', '.join(robot.communication_protocols)}
    - Processing Power: {robot.processing_power} units
    - Autonomy Level: {robot.autonomy_level}
    - Special Functions: {', '.join(robot.special_functions)}
    - Safety Features: {', '.join(robot.safety_features)}
    - Adaptability: {"Yes" if robot.adaptability else "No"}
    - Location: {robot.location[0]}, {robot.location[1]}
    - Preferred Task Types: {', '.join(robot.preferred_tasks)}

    Here are the details of the task:
    - Task Type: {task.task_type}
    - Objective: {task.objective}
    - Priority Level: {task.priority_level}
    - Reward: {task.reward} units
    - Difficulty: {task.difficulty} (scale of 1 to 10)
    - Location: {task.location} (x, y, z coordinates)
    - Navigation Constraints: {', '.join(task.navigation_constraints) if task.navigation_constraints else 'None'}
    - Required Capabilities: {', '.join(task.required_capabilities)}
    - Duration: {task.duration} seconds
    - Environmental Conditions: {', '.join(task.environmental_conditions) if task.environmental_conditions else 'None'}
    - Tools Needed: {', '.join(task.tools_needed) if task.tools_needed else 'None'}
    - Communication Requirements: {', '.join(task.communication_requirements) if task.communication_requirements else 'None'}
    - Safety Protocols: {', '.join(task.safety_protocols) if task.safety_protocols else 'None'}
    - Performance Metrics: {', '.join(task.performance_metrics) if task.performance_metrics else 'None'}
    - Success Criteria: {task.success_criteria}

    Based on the robot's capabilities and the task requirements, please rate the suitability of this robot for this task on a scale of 0 to 10, where 0 means the robot is completely unsuitable and 10 means it is perfectly suited.

    Provide a single number between 0 and 10 as the output score: 
    """

    # Send the prompt to the LLM and parse the response
#     try:
#         response = openai.Completion.create(
#             engine="text-davinci-003",  # Replace with the appropriate model
#             prompt=prompt,
#             max_tokens=5,
#             temperature=0  # Lower temperature for more deterministic responses
#         )
#         score_text = response.choices[0].text.strip()
#         score = float(score_text) if score_text.isnumeric() else 0.0
#     except Exception as e:
#         print(f"Error querying LLM: {e}")
#         score = 0.0  # Default to zero in case of an error

    return prompt

In [6]:
from itertools import combinations
def generate_random_assignments(num_robots: int, num_tasks: int, num_assignments: int = 5) -> List[List[Tuple[Optional[int], Optional[int]]]]:
    """
    Generates a list of random assignments of robots to tasks.
    Each assignment is a list of (robot, task) pairs, where some robots or tasks may be unassigned.
    
    Parameters:
    - num_robots: Number of available robots.
    - num_tasks: Number of tasks to be assigned.
    - num_assignments: Number of different random assignments to generate.
    
    Returns:
    - A list of random assignments, where each assignment is a list of (robot, task) pairs.
    """
    assignments = []

    # Generate random assignments
    for _ in range(num_assignments):
        # Generate random indices for pairing
        available_robots = random.sample(range(num_robots), num_robots)
        available_tasks = random.sample(range(num_tasks), num_tasks)
        
        # Determine the number of pairs to create (limited by the smaller of num_robots or num_tasks)
        num_pairs = min(num_robots, num_tasks)

        # Pair robots and tasks using the precomputed random indices
        assigned_pairs = [(available_robots[i], available_tasks[i]) for i in range(num_pairs)]

        # Determine unassigned robots and tasks
        unassigned_robots = available_robots[num_pairs:] if num_robots > num_pairs else []
        unassigned_tasks = available_tasks[num_pairs:] if num_tasks > num_pairs else []

        assignments.append((assigned_pairs, unassigned_robots, unassigned_tasks))

    return assignments

def generate_all_unique_assignments(num_robots: int, num_tasks: int) -> List[Tuple[List[Tuple[int, int]], List[int], List[int]]]:
    """
    Generates all possible unique assignments of robots to tasks.
    Each assignment is a tuple containing:
      1. A list of (robot, task) pairs.
      2. A list of unassigned robots.
      3. A list of unassigned tasks.
    
    Parameters:
    - num_robots: Number of available robots.
    - num_tasks: Number of tasks to be assigned.
    
    Returns:
    - A list of all possible unique assignments, where each assignment is a tuple (assigned_pairs, unassigned_robots, unassigned_tasks).
    """
    all_assignments = []

    # Determine the number of pairs to create (limited by the smaller of num_robots or num_tasks)
    num_pairs = min(num_robots, num_tasks)

    # Generate all possible combinations of robots for the tasks
    robot_combinations = list(combinations(range(num_robots), num_pairs))
    task_combinations = list(combinations(range(num_tasks), num_pairs))

    # Generate all permutations of those combinations
    for robot_comb in robot_combinations:
        for task_comb in task_combinations:
            # Create all permutations for the selected robot and task pairs
            for permuted_robots in permutations(robot_comb):
                for permuted_tasks in permutations(task_comb):
                    # Pair the robots and tasks
                    assigned_pairs = [(permuted_robots[i], permuted_tasks[i]) for i in range(num_pairs)]

                    # Determine unassigned robots and tasks
                    unassigned_robots = [r for r in range(num_robots) if r not in permuted_robots]
                    unassigned_tasks = [t for t in range(num_tasks) if t not in permuted_tasks]

                    all_assignments.append((assigned_pairs, unassigned_robots, unassigned_tasks))

    return all_assignments

def generate_high_suitability_assignments(num_robots: int, num_tasks: int, suitability_matrix: List[List[float]], num_assignments: int = 5) -> List[Tuple[List[Tuple[int, int]], List[int], List[int]]]:
    """
    Generates assignments for robots with the highest total suitability across all tasks.
    Each assignment includes the top robots paired with tasks based on suitability.
    
    Parameters:
    - num_robots: Number of available robots.
    - num_tasks: Number of tasks to be assigned.
    - suitability_matrix: A 2D list where element [i][j] represents the suitability of robot i for task j.
    - num_assignments: Number of different random assignments to generate.
    
    Returns:
    - A list of high-suitability assignments, where each assignment is a tuple containing:
      - A list of (robot, task) pairs.
      - A list of unassigned robots.
      - A list of unassigned tasks.
    """
    assignments = []
    if num_robots == 1 and num_tasks == 1:
        assigned_pairs = [(0,0)]
        unassigned_robots = []
        unassigned_tasks = []
        assignments.append((assigned_pairs, unassigned_robots, unassigned_tasks))
        return assignments
    elif num_robots == 1:
        for task_id in range(num_tasks):
            assigned_pairs = [(0, task_id)]
            unassigned_tasks = list(set(range(num_tasks)) - {task_id})
            assignments.append((assigned_pairs, [], unassigned_tasks))
        return assignments
    elif num_tasks == 1:
        for robot_id in range(num_robots):
            assigned_pairs = [(robot_id, 0)]
            unassigned_robots = list(set(range(num_robots)) - {robot_id})
            assignments.append((assigned_pairs, unassigned_robots, []))
        return assignments
        
    num_assignments = min(num_assignments, num_robots * num_tasks * 10)
    

    # Calculate total suitability score for each robot across all tasks
    robot_suitability_scores = [(i, sum(suitability_matrix[i])) for i in range(num_robots)]
    
    # Sort robots by total suitability score in descending order
    sorted_robots = sorted(robot_suitability_scores, key=lambda x: x[1], reverse=True)
    
    # Select the top robots based on total suitability scores
    top_robot_indices = [robot[0] for robot in sorted_robots[:min(num_robots, num_tasks)]]
    
    # Identify robots with the maximum score for each task
    for task in range(num_tasks):
        max_score = max(suitability_matrix[robot][task] for robot in range(num_robots))
        top_task_robots = [robot for robot in range(num_robots) if suitability_matrix[robot][task] == max_score]
        top_robot_indices.extend(top_task_robots)
    
    # Remove duplicates and keep only as many robots as tasks if robots exceed tasks
    top_robot_indices = list(set(top_robot_indices))[:min(num_robots, num_tasks)]

    for _ in range(num_assignments):
        # Randomly sample tasks to pair with the top robots
        available_tasks = random.sample(range(num_tasks), min(num_tasks, num_robots))
        
        # Randomly sample robots to pair with the tasks
        available_robots = random.sample(top_robot_indices, min(num_tasks, num_robots))

        # Pair each selected robot with a randomly chosen task
        assigned_pairs = [(available_robots[i], available_tasks[i]) for i in range(len(available_robots))]
        
        # Determine unassigned robots and tasks
        unassigned_robots = list(set(range(num_robots)) - set(available_robots))
        unassigned_tasks = list(set(range(num_tasks)) - set(available_tasks))
        # Store each assignment as a tuple
        assignments.append((assigned_pairs, unassigned_robots, unassigned_tasks))

    return assignments

In [7]:
def calculate_total_suitability(assignment: List[Tuple[int, int]], suitability_matrix: List[List[float]]) -> float:
    """
    Calculates the total suitability score for a given assignment.
    
    Parameters:
    - assignment: A list of (robot, task) pairs representing the assignment.
    - suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
    
    Returns:
    - The total suitability score for the assignment.
    """
    total_suitability = 0.0
    
    # Sum the suitability ratings for each robot-task pair in the assignment
    for robot, task in assignment:
        total_suitability += suitability_matrix[robot][task]
    
    return total_suitability

def rank_assignments_range(assignments: List[List[Tuple[int, int]]], suitability_matrix: List[List[float]]) -> Tuple[List[float], List[int]]:
    """
    Ranks the given assignments based on their total suitability scores.
    
    Parameters:
    - assignments: A list of assignments, where each assignment is a list of (robot, task) pairs.
    - suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
    
    Returns:
    - A tuple containing:
      1. A list of total suitability scores for each assignment.
      2. A list of indices representing the ranking of the assignments, where the first index corresponds to the
         assignment with the highest total score.
    """
    total_scores = []

    # Calculate total suitability scores for each assignment
    for assignment in assignments:
        total_suitability = sum(suitability_matrix[robot][task] for robot, task in assignment[0])
        total_scores.append(total_suitability)

    # Get the ranking of assignments based on the total scores (higher score is better)
    ranking = sorted(range(len(total_scores)), key=lambda i: total_scores[i], reverse=True)
    
    return total_scores, ranking

def rank_assignments_borda(assignments: List[List[Tuple[int, int]]], suitability_matrix: List[List[float]]) -> Tuple[List[float], List[int]]:
    """
    Ranks each assignment from the perspective of each robot based on its suitability for its task in each assignment,
    and uses the Borda count method to aggregate these rankings.
    
    Parameters:
    - assignments: A list of assignments, where each assignment is a list of (robot, task) pairs.
    - suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
    
    Returns:
    - A tuple containing:
      1. A list of Borda scores for each assignment.
      2. A list of indices representing the ranking of the assignments based on Borda scores, where the first index corresponds
         to the assignment with the highest total Borda score.
    """
    num_assignments = len(assignments)
    num_robots = len(suitability_matrix)

    # Initialize Borda scores for each assignment
    borda_scores = [0] * num_assignments

    # For each robot, rank the assignments based on its suitability rating
    for robot in range(num_robots):
        # Collect suitability scores for this robot across all assignments
        robot_scores = []
        
        for assignment_index, assignment in enumerate(assignments):
            # Find the task assigned to this robot in the current assignment
            assigned_task = next((task for r, task in assignment[0] if r == robot), None)
            
            # If the robot has no task assigned in this assignment, assign it the lowest score
            if assigned_task is None:
                robot_scores.append((assignment_index, -1))  # Using -1 to rank unassigned lower
            else:
                robot_scores.append((assignment_index, suitability_matrix[robot][assigned_task]))

        # Sort robot's scores for all assignments: unassigned (score -1) at the bottom, then by suitability
        robot_scores.sort(key=lambda x: x[1], reverse=True)

        # Assign Borda points based on the ranking
        for rank, (assignment_index, _) in enumerate(robot_scores):
            borda_points = len(robot_scores) - rank - 1  # Borda count: higher ranks get more points
            borda_scores[assignment_index] += borda_points

    # Rank assignments based on total Borda scores (higher score is better)
    ranked_assignments = sorted(range(num_assignments), key=lambda i: borda_scores[i], reverse=True)

    return borda_scores, ranked_assignments

def rank_assignments_approval(assignments: List[List[Tuple[int, int]]], suitability_matrix: List[List[float]], threshold: float = 10) -> Tuple[List[int], List[int]]:
    """
    Uses approval voting where each robot gives 1 point to assignments where its suitability for its task is above a threshold.
    If a robot's suitability is below the threshold or it is unassigned, it gives 0 points for that assignment.
    
    Parameters:
    - assignments: A list of assignments, where each assignment is a list of (robot, task) pairs.
    - suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
    - threshold: The suitability threshold above which a robot will approve an assignment (default is 10).
    
    Returns:
    - A tuple containing:
      1. A list of approval scores for each assignment.
      2. A list of indices representing the ranking of the assignments based on approval scores.
    """
    num_assignments = len(assignments)
    num_robots = len(suitability_matrix)

    # Initialize approval scores for each assignment
    approval_scores = [0] * num_assignments

    # For each robot, evaluate each assignment based on the suitability threshold
    for robot in range(num_robots):
        for assignment_index, assignment in enumerate(assignments):
            # Find the task assigned to this robot in the current assignment
            assigned_task = next((task for r, task in assignment[0] if r == robot), None)
            
            # Check if robot's suitability rating for this task is above the threshold
            if assigned_task is not None and suitability_matrix[robot][assigned_task] > threshold:
                approval_scores[assignment_index] += 1  # Robot approves this assignment

    # Rank assignments based on approval scores (higher score is better)
    ranked_assignments = sorted(range(num_assignments), key=lambda i: approval_scores[i], reverse=True)

    return approval_scores, ranked_assignments

def rank_assignments_majority_judgment(assignments: List[List[Tuple[int, int]]], suitability_matrix: List[List[float]]) -> Tuple[List[float], List[int]]:
    """
    Uses majority judgment to rank assignments based on robot suitability for assigned tasks.
    
    Parameters:
    - assignments: A list of assignments, where each assignment is a list of (robot, task) pairs.
    - suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
    
    Returns:
    - A tuple containing:
      1. A list of median scores for each assignment based on majority judgment.
      2. A list of indices representing the ranking of assignments based on median scores.
    """
    # Define qualitative rating categories mapped to suitability score ranges
    rating_scale = {"Excellent": 3, "Good": 2, "Fair": 1, "Poor": 0}
    rating_thresholds = [(15, "Excellent"), (10, "Good"), (5, "Fair"), (0, "Poor")]

    assignment_ratings = []

    for assignment in assignments:
        ratings = []
        
        for robot, task in assignment[0]:
            # Get the suitability score for the robot-task pair
            suitability_score = suitability_matrix[robot][task]

            # Convert suitability score to qualitative rating
            for threshold, rating in rating_thresholds:
                if suitability_score >= threshold:
                    ratings.append(rating_scale[rating])
                    break
        
        # Calculate the median rating for this assignment
        median_rating = np.median(ratings) if ratings else 0
        assignment_ratings.append(median_rating)

    # Rank assignments based on median ratings
    ranked_assignments = sorted(range(len(assignment_ratings)), key=lambda i: assignment_ratings[i], reverse=True)

    return assignment_ratings, ranked_assignments

def rank_assignments_cumulative_voting(assignments: List[List[Tuple[int, int]]], suitability_matrix: List[List[float]], total_votes: int = 10) -> Tuple[List[float], List[int]]:
    """
    Uses cumulative voting to rank assignments, where each robot distributes a fixed number of votes
    across assignments based on suitability scores.
    
    Parameters:
    - assignments: A list of assignments, where each assignment is a list of (robot, task) pairs.
    - suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
    - total_votes: The total number of votes each robot has to distribute (default is 10).
    
    Returns:
    - A tuple containing:
      1. A list of cumulative votes for each assignment.
      2. A list of indices representing the ranking of assignments based on cumulative votes.
    """
    num_assignments = len(assignments)
    cumulative_votes = [0] * num_assignments
    if total_votes < num_assignments/2:
        total_votes = num_assignments/2
    
    for robot in range(len(suitability_matrix)):
        # Collect suitability scores for this robot across all assignments
        robot_votes = []
        
        for assignment_index, assignment in enumerate(assignments):
            # Find the task assigned to this robot in the current assignment
            assigned_task = next((task for r, task in assignment[0] if r == robot), None)
            
            # If robot is unassigned in this assignment, give it a suitability score of 0
            suitability_score = suitability_matrix[robot][assigned_task] if assigned_task is not None else 0
            robot_votes.append((assignment_index, suitability_score))
        
        # Sort robot votes based on suitability scores in descending order
        robot_votes.sort(key=lambda x: x[1], reverse=True)

        # Distribute the total votes among assignments proportional to suitability scores
        total_score = sum(score for _, score in robot_votes if score > 0)
        for assignment_index, score in robot_votes:
            if total_score > 0:
                cumulative_votes[assignment_index] += total_votes * (score / total_score)

    # Rank assignments based on cumulative votes (higher score is better)
    ranked_assignments = sorted(range(num_assignments), key=lambda i: cumulative_votes[i], reverse=True)

    return cumulative_votes, ranked_assignments

def rank_assignments_condorcet_method(assignments: List[List[Tuple[int, int]]], suitability_matrix: List[List[float]]) -> Tuple[List[int], List[int]]:
    """
    Uses the Condorcet method to rank assignments by comparing each assignment in a pairwise manner.
    
    Parameters:
    - assignments: A list of assignments, where each assignment is a list of (robot, task) pairs.
    - suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
    
    Returns:
    - A tuple containing:
      1. A list of pairwise win counts for each assignment.
      2. A list of indices representing the ranking of assignments based on pairwise win counts.
    """
    num_assignments = len(assignments)
    num_robots = len(suitability_matrix)

    # Initialize a matrix to track pairwise wins for each assignment comparison
    pairwise_wins = [0] * num_assignments

    # Compare each pair of assignments
    for i in range(num_assignments):
        for j in range(i + 1, num_assignments):
            # Count the number of robots that prefer assignment i over assignment j and vice versa
            i_wins, j_wins = 0, 0
            
            for robot in range(num_robots):
                # Find the task assigned to the robot in both assignments
                task_i = next((task for r, task in assignments[i][0] if r == robot), None)
                task_j = next((task for r, task in assignments[j][0] if r == robot), None)
                
                # Get suitability scores (or 0 if unassigned)
                suitability_i = suitability_matrix[robot][task_i] if task_i is not None else 0
                suitability_j = suitability_matrix[robot][task_j] if task_j is not None else 0

                # Determine which assignment the robot prefers
                if suitability_i > suitability_j:
                    i_wins += 1
                elif suitability_j > suitability_i:
                    j_wins += 1

            # Update pairwise win counts based on robot preferences
            if i_wins > j_wins:
                pairwise_wins[i] += 1
            elif j_wins > i_wins:
                pairwise_wins[j] += 1

    # Rank assignments based on pairwise win counts
    ranked_assignments = sorted(range(num_assignments), key=lambda k: pairwise_wins[k], reverse=True)

    return pairwise_wins, ranked_assignments

def check_zero_suitability(assignment: List[Tuple[int, int]], suitability_matrix: List[List[float]]) -> bool:
    """
    Checks if any robot-task pair in the assignment has a suitability rating of 0.
    
    Parameters:
    - assignment: A list of (robot, task) pairs representing the assignment.
    - suitability_matrix: A 2D list where the element at [i][j] represents the suitability of robot i for task j.
    
    Returns:
    - True if any robot-task pair in the assignment has a suitability of 0, otherwise False.
    """
    for robot, task in assignment:
        if suitability_matrix[robot][task] == 0:
            return True  # Found a zero suitability rating
    
    return False  # No zero suitability ratings found

In [8]:
def cbba_task_allocation(suitability_matrix: List[List[float]]) -> List[Tuple[int, int]]:
    """
    Uses a two-phase Consensus-Based Bundle Algorithm (CBBA) for task allocation.
    Each robot is assigned to one task, and each task is assigned to one robot.
    
    Parameters:
    - suitability_matrix: A 2D list where the element at [i][j] represents the suitability score of robot i for task j.
    
    Returns:
    - A list of (robot, task) pairs representing the final allocation.
    """
    num_robots = len(suitability_matrix)
    num_tasks = len(suitability_matrix[0])

    # Initialize assignment variables
    robot_bundles = [[] for _ in range(num_robots)]  # Bundle of tasks for each robot
    task_bids = [-1] * num_tasks  # Highest bid for each task

    # Phase 1: Bundle Construction
    for robot in range(num_robots):
        # Each robot evaluates each task for inclusion in its bundle
        for task in range(num_tasks):
            bid = suitability_matrix[robot][task]
            if bid > task_bids[task]:
                task_bids[task] = bid
            robot_bundles[robot].append(task)
    
    not_assigned = list(range(num_robots))
    # Phase 2: Conflict Resolution
    updated = True
    while updated:
        updated = False
        
        for robot in range(num_robots):
            if not robot_bundles[robot] or robot not in not_assigned:
                continue  # Skip robots with no tasks in their bundle
                
            # Sort the robot's bundle in descending order of suitability scores
            robot_bundles[robot].sort(key=lambda task: suitability_matrix[robot][task], reverse=True)
            # Iterate through tasks in the robot's bundle
            for task in robot_bundles[robot][:]:
                # If the robot has the highest unique bid for this task, assign it
                highest_bid = max(suitability_matrix[competing_robot][task] for competing_robot in not_assigned)
                if suitability_matrix[robot][task] == highest_bid:
                    updated = True
                    task_bids[task] = suitability_matrix[robot][task]
                    # Clear remaining tasks from this robot's bundle
                    robot_bundles[robot] = [task]
                    not_assigned.remove(robot)
                    for r in range(num_robots):
                        if r != robot and task in robot_bundles[r]:
                            robot_bundles[r].remove(task)
                    break

        if len(not_assigned) == 0:
            updated = False
                    
    final_assignment = [(index, bundle[0]) for index, bundle in enumerate(robot_bundles) if bundle]

    return final_assignment

def ssia_task_allocation(suitability_matrix: List[List[float]]) -> List[Tuple[int, int]]:
    """
    Uses the Sequential Single-Item Auction (SSIA) for task allocation.
    
    Parameters:
    - suitability_matrix: A 2D list where the element at [i][j] represents the suitability score of robot i for task j.
    
    Returns:
    - A list of (robot, task) pairs representing the final allocation.
    """
    num_robots = len(suitability_matrix)
    num_tasks = len(suitability_matrix[0])
    
    # Initialize assignment list to store (robot, task) pairs
    assignments = []
    assigned_robots = set()  # Track robots that have already been assigned
    
    # Auction tasks sequentially
    for task in range(num_tasks):
        highest_bid = -1
        winning_robot = -1

        # Robots bid for the current task
        for robot in range(num_robots):
            if robot in assigned_robots:
                continue  # Skip if the robot is already assigned
            bid = suitability_matrix[robot][task]
            if bid > highest_bid:
                highest_bid = bid
                winning_robot = robot
        
        # Assign the task to the robot with the highest bid if any bid is positive
        if highest_bid > 0:
            assignments.append((winning_robot, task))
            assigned_robots.add(winning_robot)  # Mark this robot as assigned
    
    return assignments

def ilp_task_allocation(suitability_matrix: List[List[float]]) -> List[Tuple[int, int]]:
    """
    Uses Integer Linear Programming (ILP) to maximize suitability-based task allocation.
    
    Parameters:
    - suitability_matrix: A 2D list where the element at [i][j] represents the suitability score of robot i for task j.
    
    Returns:
    - A list of (robot, task) pairs representing the final assignment.
    """
    num_robots = len(suitability_matrix)
    num_tasks = len(suitability_matrix[0])

    # Define the ILP problem
    problem = pulp.LpProblem("TaskAssignment", pulp.LpMaximize)

    # Define binary decision variables x_ij where x_ij = 1 if robot i is assigned to task j, else 0
    x = [[pulp.LpVariable(f"x_{i}_{j}", cat="Binary") for j in range(num_tasks)] for i in range(num_robots)]

    # Objective: Maximize total suitability score
    problem += pulp.lpSum(suitability_matrix[i][j] * x[i][j] for i in range(num_robots) for j in range(num_tasks))

    # Constraint: Each task can be assigned to at most one robot
    for j in range(num_tasks):
        problem += pulp.lpSum(x[i][j] for i in range(num_robots)) <= 1, f"Task_{j}_Assignment"

    # Constraint: Each robot can be assigned to at most one task
    for i in range(num_robots):
        problem += pulp.lpSum(x[i][j] for j in range(num_tasks)) <= 1, f"Robot_{i}_Capacity"

    # Solve the problem
    problem.solve(pulp.PULP_CBC_CMD(msg=False))

    # Collect the results
    assignment = []
    for i in range(num_robots):
        for j in range(num_tasks):
            if pulp.value(x[i][j]) == 1:
                assignment.append((i, j))

    return assignment

In [9]:
def print_robots(robots: List[CapabilityProfile]):
    for robot in robots:
        print(robot.robot_id)
        
def print_tasks(tasks: List[TaskDescription]):
    for task in tasks:
        print(task.task_id)

def calculate_suitability_matrix(robots: List[CapabilityProfile], tasks: List[TaskDescription], suitability_method: str) -> np.ndarray:
    """
    Calculates the suitability matrix for the given robots and tasks.
    
    Parameters:
    - robots: List of robot profiles.
    - tasks: List of task descriptions.
    - suitability_method: The name of the suitability evaluation function.
    
    Returns:
    - A 2D numpy array representing the suitability scores of each robot-task pair.
    """
    num_robots = len(robots)
    num_tasks = len(tasks)
    suitability_matrix = np.zeros((num_robots, num_tasks))

    # Evaluate suitability of each robot for each task
    for i, robot in enumerate(robots):
        for j, task in enumerate(tasks):
            suitability_score = globals()[suitability_method](robot, task)
            suitability_matrix[i][j] = suitability_score
#             print(f"Suitability of {robot.robot_id} for {task.task_id}: {suitability_score:.2f}")

    # Print the suitability matrix
#     print("\nSuitability Matrix:")
#     print(suitability_matrix)
            
    return suitability_matrix

def assign_tasks_with_voting(robots: List[CapabilityProfile], tasks: List[TaskDescription], suitability_matrix: np.ndarray, num_candidates: int, voting_method: str):
    """
    Assigns tasks to robots using random assignment and ranks the assignments using the specified voting method.
    
    Parameters:
    - robots: List of robot profiles.
    - tasks: List of task descriptions.
    - suitability_matrix: A 2D numpy array with suitability scores for each robot-task pair.
    - num_candidates: Number of candidate assignments to generate.
    - voting_method: The name of the voting function.
    
    Returns:
    - The best assignment, its suitability score, and the time taken for the voting process.
    """
    num_robots = len(robots)
    num_tasks = len(tasks)

    # Display generated robots and tasks
#     print("\nGenerated Robot Profiles:")
#     for robot in robots:
#         print(robot)

#     print("\nGenerated Task Descriptions:")
#     for task in tasks:
#         print(task)
    
#     random_assignments = generate_random_assignments(num_robots, num_tasks, num_candidates)
    random_assignments = generate_high_suitability_assignments(num_robots, num_tasks, suitability_matrix, num_candidates)
#     print("\nRandom Complete Assignments of Robots to Tasks:")
#     for i, assignment in enumerate(random_assignments):
#         print(f"Assignment {i+1}: {assignment}")
#         total_score = calculate_total_suitability(assignment[0], suitability_matrix)
#         print(f"Total Suitability Score for the Assignment: {total_score:.2f}")
    
    start = time.perf_counter_ns()
    # NOTE: are assignment rankings using robot_ids from the capability profiles?
    total_scores, assignment_ranking = globals()[voting_method](random_assignments, suitability_matrix)
    end = time.perf_counter_ns()
    length = (end - start) / 1000.0
    
#     print("\nTotal Suitability Scores for Each Assignment:")
#     for i, score in enumerate(total_scores):
#         print(f"Assignment {i+1}: {score:.2f}")

#     print("\nRanking of Assignments (from best to worst):")
#     for rank, assignment_index in enumerate(assignment_ranking):
#         print(f"Rank {rank+1}: Assignment {assignment_index+1}")

    best_ranking = 0
    while(check_zero_suitability(random_assignments[assignment_ranking[best_ranking]][0], suitability_matrix) and best_ranking < len(assignment_ranking)-1):
        best_ranking += 1
    if best_ranking == num_candidates-1:
        best_ranking = 0
    
#     print(best_ranking)
#     print(random_assignments[assignment_ranking[best_ranking]])
    # best_assignment = [[(robot, task), (robot, task), ...], [unassigned robots], [unassigned tasks]]
    best_assignment = random_assignments[assignment_ranking[best_ranking]]
    best_score = calculate_total_suitability(best_assignment[0], suitability_matrix)
    # print(f"BEST ASSIGNMENT: {best_assignment}")
    # print(f"BEST ASSIGNMENT[0]: {best_assignment[0]}")

    # code to convert suitability matrix indices back to robot index, was done in main simulation instead
    # assigned_robots = {}
    # unassigned_robots = []
    # unassigned_tasks = []
    # for (robot_idx, task_idx) in best_assignment[0]:
    #     actual_robot_id = robots[robot_idx].robot_id
    #     actual_task_id = tasks[task_idx].task_id
    #     assigned_robots[actual_robot_id] = actual_task_id
    # for robot_idx in best_assignment[1]:
    #     actual_robot_id = robots[robot_idx].robot_id
    #     unassigned_robots.append(actual_robot_id)
    # for task_idx in best_assignment[2]:
    #     actual_task_id = tasks[task_idx].task_id
    #     unassigned_tasks.append(actual_task_id)

    # returns the best assignment of robots to tasks and the total suitability score of the best assignment
    return best_assignment, best_score, length

def assign_tasks_with_method(
    allocation_method: Callable[[List[List[float]]], List[Tuple[int, int]]],
    suitability_matrix: List[List[float]]
):
    """
    Assigns tasks using a specified allocation method and returns the allocation details.
    
    Parameters:
    - allocation_method: The function used for task allocation (e.g., `cbba_task_allocation`, `ssia_task_allocation`, `ilp_task_allocation`).
    - suitability_matrix: A 2D list where the element at [i][j] represents the suitability score of robot i for task j.
    
    Returns:
    - A tuple containing:
      1. A list of assigned (robot, task) pairs.
      2. A list of unassigned robot indices.
      3. A list of unassigned task indices.
      4. The total suitability score of the assignment.
      5. The time taken for the allocation (in microseconds).
    """
    num_robots = len(suitability_matrix)
    num_tasks = len(suitability_matrix[0])
    
    # Start timing
    start_time = time.perf_counter_ns()
    
    # Get the assignment using the specified allocation method
    assignment = allocation_method(suitability_matrix)
    
    # End timing
    end_time = time.perf_counter_ns()
    allocation_time = (end_time - start_time) / 1000.0  # Convert nanoseconds to microseconds
    
    # Calculate total suitability score for the assignment
    total_score = calculate_total_suitability(assignment, suitability_matrix)
    
    # Determine unassigned robots and tasks
    assigned_robots = {robot for robot, _ in assignment}
    assigned_tasks = {task for _, task in assignment}
    
    unassigned_robots = [robot for robot in range(num_robots) if robot not in assigned_robots]
    unassigned_tasks = [task for task in range(num_tasks) if task not in assigned_tasks]
    
    return (assignment, unassigned_robots, unassigned_tasks), total_score, allocation_time

In [10]:
def simulate_time_step(
    robots: List[CapabilityProfile],
    tasks: List[TaskDescription],
    unassigned_robots: List[int],
    unassigned_tasks: List[str],
    suitability_method: str,
    occupied_locations: set,
    start_positions: list,
    time_step: float = 1.0,
    total_reward: float = 0.0,
    total_success: int = 0
) -> Tuple[int, float, int]:
    """
    Simulates a single time step, updating robot positions, task progress, and handling failures.

    Parameters:
    - robots: List of all robots.
    - tasks: List of all tasks.
    - time_step: The time increment for the simulation step.
    - total_reward: Accumulated reward from successfully completed tasks.

    Returns:
    - A count of unassigned robots and the updated total reward.
    """
    count = 0
    for robot in robots:
#         print(robot.robot_id)
        if robot.current_task is not None:
            # Get the assigned task
            task = robot.current_task
#             print(task.task_id, robot.remaining_distance, robot.time_on_task, task.time_to_complete)

            # check if there is more path to traverse for the robot
            if robot.current_path and len(robot.current_path) > 1:
                # move one step along the path by removing the current position                
                # NOTE: Speed isnt going to work with CBS as it currently is
                # 1. Pick a max speed (15 according to random_capability_profile) to represent the number of sub-time steps to make up a single macro time step
                # 2. replace each single time step with 15 sub steps instead of one big step
                # 3. Now a robot with speed v (where v <= 15) can move up to v steps (or cells) within those 15 sub steps
                # 4. within each sub-step, a robot will either move or stay still, if it has a speed of 5, it can only move 5 times during the 15 sub-steps (speed will have to be integers)

                # NOTE: MAKE ROBOTS CONSTANT SPEED 
                # NOTE: DONT MOVE CBS HERE, THIS IS WHERE MOVEMENT SHOULD BE
                # NOTE: must update occupied_locations here

                next_position = robot.current_path[1] # gives (x, y) coordinate of next step in path
                occupied_locations.discard((robot.location[0], robot.location[1])) # remove current location from occupied set
                robot.location = (next_position[0], next_position[1]) # update location
                # start position for this robot should be replaced, if not then must index by ID
                start_positions[robot] = robot.location
                occupied_locations.add((next_position[0], next_position[1])) # update occupied set
                robot.current_path.pop(0) # Move the robot one space
                robot.remaining_distance = len(robot.current_path) - 1 # recalculate the remaining distance and new location

                if robot.remaining_distance <= 1:
                    # The robot has reached the task
                    robot.remaining_distance = 0
                    robot.time_on_task = 0  # Reset time on task so it can begin work (time on task is a counter for how long it takes to complete a task)
                    
                if random.random() < 0.001:  # Example 0.1% failure rate during navigation
#                     print(f"Robot {robot.robot_id} failed to reach task {task.task_id} due to mechanical failure.")
                    # get task id and task index
                    task_id = robot.current_task.task_id
                    t_index = [task.task_id for task in tasks ].index(task_id)
                    # unassign task
                    tasks[t_index].assigned_robot = None
                    # move it to unassigned tasks list
                    unassigned_tasks.append(tasks[t_index].task_id)
                    # unassign robot
                    robot.current_task = None
                    # move it to unassigned robots list
                    unassigned_robots.append(robot.robot_id)
                    # NOTE: may need to make robot.current_path an empty list when unassigned to prevent unintentional movement
                    continue

            # If the robot is at the task, increment time on task
            if robot.remaining_distance == 0:
                robot.time_on_task += time_step
                suitability = globals()[suitability_method](robot, task)
                failure_probability = 1 / (100 * (suitability + 1))  # Higher suitability, lower failure rate
                # Check if the task is completed
                if robot.time_on_task >= task.time_to_complete:
                    # Mark task as completed
#                     print_robots(robots)
#                     print_tasks(tasks)
                    total_reward += task.reward
                    total_success += 1
                    robot.current_task = None
                    robot.tasks_successful += 1
                    unassigned_robots.append(robot.robot_id)
                    # NOTE: may need to make robot.current_path an empty list when unassigned to prevent unintentional movement
#                     print(f"Robot {robot.robot_id} completed task {task.task_id}.")
                    tasks.remove(task)
                elif random.random() < failure_probability:
#                     print(f"Robot {robot.robot_id} failed task {task.task_id} during execution.")
                    task_id = robot.current_task.task_id
                    t_index = [task.task_id for task in tasks ].index(task_id)
                    tasks[t_index].assigned_robot = None
                    unassigned_tasks.append(tasks[t_index].task_id)
                    robot.current_task = None
                    unassigned_robots.append(robot.robot_id)
                    # NOTE: may need to make robot.current_path an empty list when unassigned to prevent unintentional movement
                    continue
        else:
            count += 1
    return count, total_reward, total_success

def add_new_tasks(tasks: List[TaskDescription], unassigned_tasks: List[str], task_max_id: int, new_task_count: int, total_tasks: int, grid: List[List[int]], occupied_locations: set) -> Tuple[int, int]:
    """Adds new unassigned tasks to the system."""
    for _ in range(new_task_count):
        task_id = f"T{task_max_id}"
        task_max_id += 1
        total_tasks += 1
        new_task = generate_random_task_description(task_id, grid, occupied_locations)
        tasks.append(new_task)
        unassigned_tasks.append(task_id)
#         print(f"New task {new_task.task_id} added.")
    return task_max_id, total_tasks

def add_new_robots(robots: List[CapabilityProfile], unassigned_robots: List[str], robot_max_id: int, new_robot_count: int, grid: List[List[int]], occupied_locations: set) -> int:
    """Adds new UNASSIGNED robots to the system."""
    for _ in range(new_robot_count):
        robot_id = f"R{robot_max_id}"
        robot_max_id += 1
        new_robot = generate_random_robot_profile(robot_id, grid, occupied_locations)
        robots.append(new_robot)
        unassigned_robots.append(robot_id)
        # NOTE: may need to make robot.current_path an empty list when unassigned to prevent unintentional movement
#         print(f"New robot {new_robot.robot_id} added.")
    return robot_max_id

def remove_random_robots(robots: List[CapabilityProfile], tasks: List[TaskDescription], unassigned_robots: List[str], unassigned_tasks: List[str], count: int, occupied_locations: set):
    """Randomly removes robots from the system."""
    for _ in range(min(count, len(robots))):
        robot_to_remove = random.choice(robots)
        occupied_locations.discard((robot_to_remove.location[0], robot_to_remove.location[1]))
        if robot_to_remove.current_task == None:
            unassigned_robots.remove(robot_to_remove.robot_id)
        else:
            task_id = robot_to_remove.current_task.task_id
            # NOTE: try this method below for searching the robot and task indices
            t_index = [task.task_id for task in tasks ].index(task_id)
            tasks[t_index].assigned_robot = None
            unassigned_tasks.append(tasks[t_index].task_id)
        robots.remove(robot_to_remove)
#         print(f"Robot {robot_to_remove.robot_id} left the system. It attempted {robot_to_remove.tasks_attempted} tasks and successfully completed {robot_to_remove.tasks_successful} of them.")

def reassign_robots_to_tasks(robots: List[CapabilityProfile], tasks: List[TaskDescription], num_candidates: int, voting_method: str, suitability_method: str, unassigned_robots: List[str], unassigned_tasks: List[str]):
    """Tries to assign unassigned robots to unassigned tasks."""
#     print(unassigned_robots, unassigned_tasks)
    urobots = [robot for robot in robots if robot.robot_id in unassigned_robots]
    utasks = [task for task in tasks if task.task_id in unassigned_tasks]
    suitability_matrix = calculate_suitability_matrix(urobots, utasks, suitability_method)
    output, score, length = assign_tasks_with_voting(urobots, utasks, suitability_matrix, num_candidates, voting_method)
    assigned_pairs = output[0]
    unassigned_robots = [urobots[val].robot_id for val in output[1]]
    unassigned_tasks = [utasks[val].task_id for val in output[2]]
    for (robot_idx, task_idx) in assigned_pairs:
        robots[robot_idx].current_task = tasks[task_idx]
        # NOTE: leave remaining distance for the CBS call to create path then the simulate time step call will calculate it (its just path length) 
        robots[robot_idx].tasks_attempted += 1
        tasks[task_idx].assigned_robot = robots[robot_idx]
    return unassigned_robots, unassigned_tasks, score, length

def reassign_robots_to_tasks_with_method(robots: List[CapabilityProfile], tasks: List[TaskDescription], num_candidates: int, voting_method: str, suitability_method: str, unassigned_robots: List[str], unassigned_tasks: List[str], allocation_method: Callable[[List[List[float]]], List[Tuple[int, int]]]):
    """Tries to assign unassigned robots to unassigned tasks."""
#     print(unassigned_robots, unassigned_tasks)
    urobots = [robot for robot in robots if robot.robot_id in unassigned_robots]
    utasks = [task for task in tasks if task.task_id in unassigned_tasks]
    suitability_matrix = calculate_suitability_matrix(urobots, utasks, suitability_method)
    output, score, length = assign_tasks_with_method(allocation_method, suitability_matrix)
    assigned_pairs = output[0]
    unassigned_robots = [urobots[val].robot_id for val in output[1]]
    unassigned_tasks = [utasks[val].task_id for val in output[2]]
    for (robot_idx, task_idx) in assigned_pairs:
        robots[robot_idx].current_task = tasks[task_idx]
        # NOTE: leave remaining distance for the CBS call to create path then the simulate time step call will calculate it (its just path length) 
        robots[robot_idx].tasks_attempted += 1
        tasks[task_idx].assigned_robot = robots[robot_idx]
    return unassigned_robots, unassigned_tasks, score, length

In [11]:
# import multiprocessing
import time
import psutil
# CBS block

def main_simulation(num_robots: int, num_tasks: int, num_candidates: int, voting_method: str, suitability_method: str, max_time_steps: int, add_tasks: bool, add_robots: bool, remove_robots: bool, tasks_to_add: int = 1, robots_to_add: int = 1, robots_to_remove: int = 1):
    # Load the MAPF benchmark map
    map_file = r"C:\Users\owner\Documents\PhD\TierLab\VBTA\MAPF_benchmark_maps\den001d.map"
    grid = load_map(map_file) # 2D list of 0/1 representing the map

    # get initial positions list to reuse later
    initial_positions = set()

    # generate_random_robot_profile now takes in the map grid and occupied positions and passes them to get_random_free_position function
    robots = [generate_random_robot_profile(f"R{i+1}", grid, initial_positions) for i in range(num_robots)]
    tasks = [generate_random_task_description(f"T{i+1}", grid, initial_positions) for i in range(num_tasks)]
    initial_robots = copy.deepcopy(robots)
    initial_tasks = copy.deepcopy(tasks)
    robot_max_id = len(robots)+1
    task_max_id = len(tasks)+1
    total_reward = 0.0
    total_success = 0.0
    total_tasks = num_tasks
    total_reassignment_time = 0.0
    total_reassignment_score = 0.0

    
    # then propogate the initial occupied positions
    for r in robots:
        # r.location is [(row, col)]
        # print(f"ROW: {r.location[0][0]}")
        # print(f"COL: {r.location[0][1]}")
        print(r.robot_id)
        row = int(r.location[0][0])
        col = int(r.location[0][1])
        initial_positions.add((row, col))

    # copy the initial positions set to use over and over
    occupied_positions = set(initial_positions) # use the occcupied positions as the current positions for CBS, this is just as a occupation check, not a start position
    
    # # testing occupied positions access for low_level_search map slot verification (empty or not)
    # print(f"OCCUPIED POSITIONS: {occupied_positions}")
    # for r in robots:
    #     row = int(r.location[0][0])
    #     col = int(r.location[0][1])
    #     if (row, col) in occupied_positions:
    #         print("TRUE")

    # Calculate suitability matrix
    suitability_matrix = calculate_suitability_matrix(robots, tasks, suitability_method)
    
    # Perform task assignment using voting
    output, score, length = assign_tasks_with_voting(robots, tasks, suitability_matrix, num_candidates, voting_method)
    
    assigned_pairs = output[0] # list of [(robot, task), (robot, task), ...] assignments NOTE: are they the correct robot_ids from the capability profiles?
    unassigned_robots = [robots[val].robot_id for val in output[1]]
    unassigned_tasks = [tasks[val].task_id for val in output[2]]
    assigned_robots = {}
    for (robot_idx, task_idx) in assigned_pairs:
        assigned_robots[robots[robot_idx].robot_id] = tasks[task_idx].task_id
#     print(unassigned_robots, unassigned_tasks, task_max_id)
#     print(num_robots, num_tasks, num_candidates, voting_method, suitability_method, score, length)
    print(f"ASSIGNED PAIRS: {assigned_pairs}")
    print(f"ASSIGNED ROBOTS: {assigned_robots}")

    cbba_output, cbba_score, cbba_length = assign_tasks_with_method(cbba_task_allocation,suitability_matrix)
    ssia_output, ssia_score, ssia_length = assign_tasks_with_method(ssia_task_allocation,suitability_matrix)
    ilp_output, ilp_score, ilp_length = assign_tasks_with_method(ilp_task_allocation,suitability_matrix)
    
    print(num_robots, num_tasks, num_candidates, voting_method, suitability_method, score, length, cbba_score, cbba_length, ssia_score, ssia_length, ilp_score, ilp_length)

    # NOTE: spot where we want to run pathfinding
    # start_positions = list(initial_positions)
    start_positions = {} # start and goal positions need to be dictionaries so we can access the corresponding robot and never get things mixed up
    goal_positions = {}
    robot_ids = []
    id_to_index = {}
    id_to_task_index = {}

    for robot in robots:
        print(f"ROBOT X LOCATION: {robot.location[0][0]}")
        print(f"ROBOT Y LOCATION: {robot.location[0][1]}")
        start_positions[robot.robot_id] = (robot.location[0][0], robot.location[0][1])
    print(f"START POSITIONS: {start_positions}")

    # change the suitability matrix indices into regular indices and then get the indices of all tasks and robots to use later
    # NOTE: fix, its gonna slow things down a lot, try method from remove random robot
    for index, robot_profile in enumerate(robots):
        id_to_index[robot_profile.robot_id] = index
    print(f"ID TO INDEX: {id_to_index}")
    # NOTE: fix, its gonna slow things down a lot
    for task_index, task_profile in enumerate(tasks):
        id_to_task_index[task_profile.task_id] = task_index
    print(f"ID TO TASK INDEX: {id_to_task_index}")

    # Better way to get indices, from remove_random_robot method

    for r_id, t_id in assigned_robots.items():
        print(f"ROBOT: {r_id}")
        print(f"TASK: {t_id}")
        # print(f"ALL ROBOTS: {robots}")
        # print(f"ALL TASKS: {tasks}")
        # print(f"ASSIGNED ROBOTS: {assigned_robots}")
        r_index = id_to_index[r_id]
        t_index = id_to_task_index[t_id]
        print(f"INDEX: {r_index}")
        # print(f"ROBOT INFO: {robots[r_index]}")
        # assign the current task
        robots[r_index].current_task = tasks[task_index]
        robots[r_index].tasks_attempted = 1
        tasks[task_index].assigned_robot = robots[r_index]
        goal_positions[r_index] = (robots[r_index].current_task.location[0][0], robots[r_index].current_task.location[0][1])
        # goal_positions[robot] = (robots[robot].current_task.location[0][0], robots[robot].current_task.location[0][1])

    # for (robot_idx, task_idx) in assigned_pairs:
    #     start_pos = robots[robot_idx].location    # (row, col) of robot initially
        # goal_pos = tasks[task_idx].location       # (row, col) of task, robot goal
    #     start_positions.append(start_pos)
        # goal_positions.append(goal_pos)


    start_robot_ids = sorted(start_positions.keys()) # keep a stable list of robot IDS so we can pass in the start and goal positions as regular lists to CBS and it will perform normally
    goal_robot_ids = sorted(goal_positions.keys()) # keep a stable list of robot IDS

    start_positions_list = [start_positions[srob_id] for srob_id in start_robot_ids]
    goal_positions_list = [goal_positions[grob_id] for grob_id in goal_robot_ids]

    # initial CBS paths done once here, then updated and run in simulate_time_step when paths need updating and robots are removed 
    solution = cbs(start_positions_list, goal_positions_list, grid, occupied_positions)

    if solution is None:
        print("CBS could not find a conflict free path assignment for all agents")
        # could possibly fall back on the simple method here if we get a lot of issues
    else:
        # Store the path in each robot
        # for robot in assigned_robots:
        # for i, (robot_idx, task_idx) in enumerate(robot_ids):
        for i, robot_indx in enumerate(start_robot_ids):
            # since CBS was calculated in this order we need to store the solution paths in this order, list of locations representing path
            path = solution[i]
            r_index = id_to_index[robot_indx]
            # set robot current task to task it is paired with in assigned pairs
            # robots[robot_idx].current_task = tasks[task_idx]
            # set robot current path to the corresponding CBS solution path
            robots[r_index].current_path = path
            # set robot remaining distance to length of the path - 1
            robots[r_index].remaining_distance = len(path) - 1
            # set robot task attempted counter
            # robots[robot_idx].tasks_attempted = 1
            # set task assigned robot to the robot it is paired with in assigned pairs
            # tasks[task_idx].assigned_robot = robots[robot_idx]

    # for pair in assigned_pairs:
    #     robots[pair[0]].current_task = tasks[pair[1]]
    #     # NOTE: this will be path length from pathfinding, how much of the path is left to get to task
    #     robots[pair[0]].remaining_distance = ((robots[pair[0]].location[0] - tasks[pair[1]].location[0]) ** 2 + (robots[pair[0]].location[1] - tasks[pair[1]].location[1]) ** 2) ** 0.5
    #     robots[pair[0]].tasks_attempted = 1
    #     tasks[pair[1]].assigned_robot = robots[pair[0]]
    
    for time_step in range(max_time_steps):
#         print(f"\n--- Time Step {time_step + 1} ---")

        # Simulate time step
        unassigned_count, total_reward, total_success = simulate_time_step(robots, tasks, unassigned_robots, unassigned_tasks, suitability_method, occupied_positions, 1.0, total_reward, total_success)
#         print(unassigned_count, len(robots), unassigned_robots, unassigned_tasks)
        # Periodically add new tasks and robots
        if add_tasks:
            task_max_id, total_tasks = add_new_tasks(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks, grid, occupied_positions)
        if add_robots:
            robot_max_id = add_new_robots(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add), grid, occupied_positions)

        # Periodically remove robots
        if remove_robots:
            remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove), occupied_positions)
            

        # Reassign unassigned robots to unassigned tasks
        if len(unassigned_robots) > 0 and len(unassigned_tasks) > 0:        # NOTE: NEED TO REPLAN PATHS USING CBS IF NEW ROBOT GETS NEW TASK
            unassigned_robots, unassigned_tasks, reassign_score, reassign_length = reassign_robots_to_tasks(robots, tasks, num_candidates, voting_method, suitability_method, unassigned_robots, unassigned_tasks)
            total_reassignment_time += reassign_length
            total_reassignment_score += reassign_score
        
        # NOTE: use occupied positions as current positions for running CBS, update goal positions whenever needed
        solution = cbs(start_positions, goal_positions, grid, occupied_positions)

    overall_success_rate = total_success / total_tasks
    print(f"Voting: Total reward: {total_reward}, Overall success rate: {overall_success_rate:.2%}, Tasks completed: {total_success}, Reassignment Time: {total_reassignment_time}, Reassignment Score: {total_reassignment_score}")
#     for robot in robots:
#         print(f"Robot {robot.robot_id} attempted {robot.tasks_attempted} tasks and successfully completed {robot.tasks_successful} of them.")

#     robots = copy.deepcopy(initial_robots)
#     tasks = copy.deepcopy(initial_tasks)
#     robot_max_id = len(robots)+1
#     task_max_id = len(tasks)+1
#     total_reward = 0.0
#     total_success = 0.0
#     total_tasks = num_tasks
#     total_reassignment_time = 0.0
#     total_reassignment_score = 0.0
#     assigned_pairs = cbba_output[0]
#     unassigned_robots = [robots[val].robot_id for val in cbba_output[1]]
#     unassigned_tasks = [tasks[val].task_id for val in cbba_output[2]]
    
#     for pair in assigned_pairs:
#         robots[pair[0]].current_task = tasks[pair[1]]
#         robots[pair[0]].remaining_distance = ((robots[pair[0]].location[0] - tasks[pair[1]].location[0]) ** 2 + (robots[pair[0]].location[1] - tasks[pair[1]].location[1]) ** 2) ** 0.5
#         robots[pair[0]].tasks_attempted = 1
#         tasks[pair[1]].assigned_robot = robots[pair[0]]
    
#     for time_step in range(max_time_steps):
# #         print(f"\n--- Time Step {time_step + 1} ---")

#         # Simulate time step
#         unassigned_count, total_reward, total_success = simulate_time_step(robots, tasks, unassigned_robots, unassigned_tasks, suitability_method, 1.0, total_reward, total_success)
# #         print(unassigned_count, len(robots), unassigned_robots, unassigned_tasks)
#         # Periodically add new tasks and robots
#         if add_tasks:
#             task_max_id, total_tasks = add_new_tasks(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks, grid)
#         if add_robots:
#             robot_max_id = add_new_robots(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add), grid, occupied_positions)

#         # Periodically remove robots
#         if remove_robots:
#             remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove))

#         # Reassign unassigned robots to unassigned tasks
#         if len(unassigned_robots) > 0 and len(unassigned_tasks) > 0:        # NOTE: NEED TO REPLAN PATHS USING CBS IF NEW ROBOT GETS NEW TASK
#             unassigned_robots, unassigned_tasks, reassign_score, reassign_length = reassign_robots_to_tasks_with_method(robots, tasks, num_candidates, voting_method, suitability_method, unassigned_robots, unassigned_tasks, cbba_task_allocation)
#             total_reassignment_time += reassign_length
#             total_reassignment_score += reassign_score

#     overall_success_rate = total_success / total_tasks
#     print(f"CBBA: Total reward: {total_reward}, Overall success rate: {overall_success_rate:.2%}, Tasks completed: {total_success}, Reassignment Time: {total_reassignment_time}, Reassignment Score: {total_reassignment_score}")
# #     for robot in robots:
# #         print(f"Robot {robot.robot_id} attempted {robot.tasks_attempted} tasks and successfully completed {robot.tasks_successful} of them.")

#     robots = copy.deepcopy(initial_robots)
#     tasks = copy.deepcopy(initial_tasks)
#     robot_max_id = len(robots)+1
#     task_max_id = len(tasks)+1
#     total_reward = 0.0
#     total_success = 0.0
#     total_tasks = num_tasks
#     total_reassignment_time = 0.0
#     total_reassignment_score = 0.0
#     assigned_pairs = ssia_output[0]
#     unassigned_robots = [robots[val].robot_id for val in ssia_output[1]]
#     unassigned_tasks = [tasks[val].task_id for val in ssia_output[2]]
    
#     for pair in assigned_pairs:
#         robots[pair[0]].current_task = tasks[pair[1]]
#         robots[pair[0]].remaining_distance = ((robots[pair[0]].location[0] - tasks[pair[1]].location[0]) ** 2 + (robots[pair[0]].location[1] - tasks[pair[1]].location[1]) ** 2) ** 0.5
#         robots[pair[0]].tasks_attempted = 1
#         tasks[pair[1]].assigned_robot = robots[pair[0]]
    
#     for time_step in range(max_time_steps):
# #         print(f"\n--- Time Step {time_step + 1} ---")

#         # Simulate time step
#         unassigned_count, total_reward, total_success = simulate_time_step(robots, tasks, unassigned_robots, unassigned_tasks, suitability_method, 1.0, total_reward, total_success)
# #         print(unassigned_count, len(robots), unassigned_robots, unassigned_tasks)
#         # Periodically add new tasks and robots
#         if add_tasks:
#             task_max_id, total_tasks = add_new_tasks(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks, grid)
#         if add_robots:
#             robot_max_id = add_new_robots(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add), grid, occupied_positions)

#         # Periodically remove robots
#         if remove_robots:
#             remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove))

#         # Reassign unassigned robots to unassigned tasks
#         if len(unassigned_robots) > 0 and len(unassigned_tasks) > 0:        # NOTE: NEED TO REPLAN PATHS USING CBS IF NEW ROBOT GETS NEW TASK
#             unassigned_robots, unassigned_tasks, reassign_score, reassign_length = reassign_robots_to_tasks_with_method(robots, tasks, num_candidates, voting_method, suitability_method, unassigned_robots, unassigned_tasks, ssia_task_allocation)
#             total_reassignment_time += reassign_length
#             total_reassignment_score += reassign_score

#     overall_success_rate = total_success / total_tasks
#     print(f"SSIA: Total reward: {total_reward}, Overall success rate: {overall_success_rate:.2%}, Tasks completed: {total_success}, Reassignment Time: {total_reassignment_time}, Reassignment Score: {total_reassignment_score}")
# #     for robot in robots:
# #         print(f"Robot {robot.robot_id} attempted {robot.tasks_attempted} tasks and successfully completed {robot.tasks_successful} of them.")

#     robots = copy.deepcopy(initial_robots)
#     tasks = copy.deepcopy(initial_tasks)
#     robot_max_id = len(robots)+1
#     task_max_id = len(tasks)+1
#     total_reward = 0.0
#     total_success = 0.0
#     total_tasks = num_tasks
#     total_reassignment_time = 0.0
#     total_reassignment_score = 0.0
#     assigned_pairs = ilp_output[0]
#     unassigned_robots = [robots[val].robot_id for val in ilp_output[1]]
#     unassigned_tasks = [tasks[val].task_id for val in ilp_output[2]]
    
#     for pair in assigned_pairs:
#         robots[pair[0]].current_task = tasks[pair[1]]
#         robots[pair[0]].remaining_distance = ((robots[pair[0]].location[0] - tasks[pair[1]].location[0]) ** 2 + (robots[pair[0]].location[1] - tasks[pair[1]].location[1]) ** 2) ** 0.5
#         robots[pair[0]].tasks_attempted = 1
#         tasks[pair[1]].assigned_robot = robots[pair[0]]
    
#     for time_step in range(max_time_steps):
# #         print(f"\n--- Time Step {time_step + 1} ---")

#         # Simulate time step
#         unassigned_count, total_reward, total_success = simulate_time_step(robots, tasks, unassigned_robots, unassigned_tasks, suitability_method, 1.0, total_reward, total_success)
# #         print(unassigned_count, len(robots), unassigned_robots, unassigned_tasks)
#         # Periodically add new tasks and robots
#         if add_tasks:
#             task_max_id, total_tasks = add_new_tasks(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks, grid)
#         if add_robots:
#             robot_max_id = add_new_robots(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add), grid, occupied_positions)

#         # Periodically remove robots
#         if remove_robots:
#             remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove))

#         # Reassign unassigned robots to unassigned tasks
#         if len(unassigned_robots) > 0 and len(unassigned_tasks) > 0:        # NOTE: NEED TO REPLAN PATHS USING CBS IF NEW ROBOT GETS NEW TASK
#             unassigned_robots, unassigned_tasks, reassign_score, reassign_length = reassign_robots_to_tasks_with_method(robots, tasks, num_candidates, voting_method, suitability_method, unassigned_robots, unassigned_tasks, ilp_task_allocation)
#             total_reassignment_time += reassign_length
#             total_reassignment_score += reassign_score

#     overall_success_rate = total_success / total_tasks
#     print(f"ILP: Total reward: {total_reward}, Overall success rate: {overall_success_rate:.2%}, Tasks completed: {total_success}, Reassignment Time: {total_reassignment_time}, Reassignment Score: {total_reassignment_score}")
# #     for robot in robots:
# #         print(f"Robot {robot.robot_id} attempted {robot.tasks_attempted} tasks and successfully completed {robot.tasks_successful} of them.")

def benchmark_simulation(num_robots: int, num_tasks: int, num_candidates: int, voting_method: str, suitability_method: str, max_time_steps: int, add_tasks: bool, add_robots: bool, remove_robots: bool, tasks_to_add: int = 1, robots_to_add: int = 1, robots_to_remove: int = 1):
    start_time = time.time()
    main_simulation(num_robots, num_tasks, num_candidates, voting_method, suitability_method, max_time_steps, add_tasks, add_robots, remove_robots, tasks_to_add, robots_to_add, robots_to_remove)
    end_time = time.time()
    execution_time = end_time - start_time

    cpu_usage = psutil.cpu_percent()
    memory_usage = psutil.virtual_memory().used

    print(f"Simulation completed in {execution_time:.2f} seconds.")
    print(f"CPU Usage: {cpu_usage}%")
    print(f"Memory Usage: {memory_usage / (1024 * 1024)} MB")

if __name__ == "__main__":
#     voting_methods = ["rank_assignments_borda", "rank_assignments_approval", "rank_assignments_majority_judgment", "rank_assignments_cumulative_voting", "rank_assignments_condorcet_method", "rank_assignments_range"]
    voting_methods = ["rank_assignments_range"]
#     suitability_methods = ["evaluate_suitability_loose", "evaluate_suitability_strict", "evaluate_suitability_distance", "evaluate_suitability_priority"]
    suitability_methods = ["evaluate_suitability_loose","evaluate_suitability_distance"]
    max_time_steps = 1000
    add_tasks = True
    add_robots = True
    remove_robots = True
    # param_combinations = []
    for i in [10]:
        for j in [10]:
            for nc in [50]:
                for vm in voting_methods:
                    for sm in suitability_methods:
                        for k in range(0,10):
                            # param_combinations.append((i, j, nc, vm, sm, max_time_steps, add_tasks, add_robots, remove_robots, 10, 10, 10))
    # with multiprocessing.Pool() as pool:
        # pool.starmap(main_simulation, param_combinations)
                            # main_simulation(i, j, nc, vm, sm, max_time_steps, add_tasks, add_robots, remove_robots,10,10,10)
                            benchmark_simulation(i, j, nc, vm, sm, max_time_steps, add_tasks, add_robots, remove_robots,10,10,10)

R1
R2
R3
R4
R5
R6
R7
R8
R9
R10
ASSIGNED PAIRS: [(1, 7), (4, 1), (7, 8), (0, 6), (2, 5), (5, 9), (3, 4), (9, 2), (8, 3), (6, 0)]
ASSIGNED ROBOTS: {'R2': 'T8', 'R5': 'T2', 'R8': 'T9', 'R1': 'T7', 'R3': 'T6', 'R6': 'T10', 'R4': 'T5', 'R10': 'T3', 'R9': 'T4', 'R7': 'T1'}
10 10 50 rank_assignments_range evaluate_suitability_loose 125.02877155205198 80.1 130.02877155205198 128.1 124.52877155205194 11.4 132.02877155205198 27107.6
ROBOT X LOCATION: 28
ROBOT Y LOCATION: 73
ROBOT X LOCATION: 33
ROBOT Y LOCATION: 14
ROBOT X LOCATION: 66
ROBOT Y LOCATION: 114
ROBOT X LOCATION: 35
ROBOT Y LOCATION: 68
ROBOT X LOCATION: 29
ROBOT Y LOCATION: 107
ROBOT X LOCATION: 20
ROBOT Y LOCATION: 88
ROBOT X LOCATION: 57
ROBOT Y LOCATION: 182
ROBOT X LOCATION: 32
ROBOT Y LOCATION: 118
ROBOT X LOCATION: 41
ROBOT Y LOCATION: 50
ROBOT X LOCATION: 49
ROBOT Y LOCATION: 106
START POSITIONS: {'R1': (28, 73), 'R2': (33, 14), 'R3': (66, 114), 'R4': (35, 68), 'R5': (29, 107), 'R6': (20, 88), 'R7': (57, 182), 'R8': (32, 118)

KeyboardInterrupt: 

In [11]:
# import multiprocessing

def main_simulation(num_robots: int, num_tasks: int, num_candidates: int, voting_method: str, suitability_method: str, max_time_steps: int, add_tasks: bool, add_robots: bool, remove_robots: bool, tasks_to_add: int = 1, robots_to_add: int = 1, robots_to_remove: int = 1):
    robots = [generate_random_robot_profile(f"R{i+1}") for i in range(num_robots)]
    tasks = [generate_random_task_description(f"T{i+1}") for i in range(num_tasks)]
    initial_robots = copy.deepcopy(robots)
    initial_tasks = copy.deepcopy(tasks)
    robot_max_id = len(robots)+1
    task_max_id = len(tasks)+1
    total_reward = 0.0
    total_success = 0.0
    total_tasks = num_tasks
    total_reassignment_time = 0.0
    total_reassignment_score = 0.0

    # Calculate suitability matrix
    suitability_matrix = calculate_suitability_matrix(robots, tasks, suitability_method)
    
    # Perform task assignment using voting
    output, score, length = assign_tasks_with_voting(robots, tasks, suitability_matrix, num_candidates, voting_method)
        
    assigned_pairs = output[0]
    unassigned_robots = [robots[val].robot_id for val in output[1]]
    unassigned_tasks = [tasks[val].task_id for val in output[2]]
#     print(unassigned_robots, unassigned_tasks, task_max_id)
#     print(num_robots, num_tasks, num_candidates, voting_method, suitability_method, score, length)
#     print(assigned_pairs)

    cbba_output, cbba_score, cbba_length = assign_tasks_with_method(cbba_task_allocation,suitability_matrix)
    ssia_output, ssia_score, ssia_length = assign_tasks_with_method(ssia_task_allocation,suitability_matrix)
    ilp_output, ilp_score, ilp_length = assign_tasks_with_method(ilp_task_allocation,suitability_matrix)
    
    print(num_robots, num_tasks, num_candidates, voting_method, suitability_method, score, length, cbba_score, cbba_length, ssia_score, ssia_length, ilp_score, ilp_length)

    # NOTE: spot where we want to run pathfinding
    for pair in assigned_pairs:
        robots[pair[0]].current_task = tasks[pair[1]]
        # NOTE: this will be path length from pathfinding, how much of the path is left to get to task
        robots[pair[0]].remaining_distance = ((robots[pair[0]].location[0] - tasks[pair[1]].location[0]) ** 2 + (robots[pair[0]].location[1] - tasks[pair[1]].location[1]) ** 2) ** 0.5
        robots[pair[0]].tasks_attempted = 1
        tasks[pair[1]].assigned_robot = robots[pair[0]]
    
    for time_step in range(max_time_steps):
#         print(f"\n--- Time Step {time_step + 1} ---")

        # Simulate time step
        unassigned_count, total_reward, total_success = simulate_time_step(robots, tasks, unassigned_robots, unassigned_tasks, suitability_method, 1.0, total_reward, total_success)
#         print(unassigned_count, len(robots), unassigned_robots, unassigned_tasks)
        # Periodically add new tasks and robots
        if add_tasks:
            task_max_id, total_tasks = add_new_tasks(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks)
        if add_robots:
            robot_max_id = add_new_robots(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add))

        # Periodically remove robots
        if remove_robots:
            remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove))

        # Reassign unassigned robots to unassigned tasks
        if len(unassigned_robots) > 0 and len(unassigned_tasks) > 0:
            unassigned_robots, unassigned_tasks, reassign_score, reassign_length = reassign_robots_to_tasks(robots, tasks, num_candidates, voting_method, suitability_method, unassigned_robots, unassigned_tasks)
            total_reassignment_time += reassign_length
            total_reassignment_score += reassign_score

    overall_success_rate = total_success / total_tasks
    print(f"Voting: Total reward: {total_reward}, Overall success rate: {overall_success_rate:.2%}, Tasks completed: {total_success}, Reassignment Time: {total_reassignment_time}, Reassignment Score: {total_reassignment_score}")
#     for robot in robots:
#         print(f"Robot {robot.robot_id} attempted {robot.tasks_attempted} tasks and successfully completed {robot.tasks_successful} of them.")

    robots = copy.deepcopy(initial_robots)
    tasks = copy.deepcopy(initial_tasks)
    robot_max_id = len(robots)+1
    task_max_id = len(tasks)+1
    total_reward = 0.0
    total_success = 0.0
    total_tasks = num_tasks
    total_reassignment_time = 0.0
    total_reassignment_score = 0.0
    assigned_pairs = cbba_output[0]
    unassigned_robots = [robots[val].robot_id for val in cbba_output[1]]
    unassigned_tasks = [tasks[val].task_id for val in cbba_output[2]]
    
    for pair in assigned_pairs:
        robots[pair[0]].current_task = tasks[pair[1]]
        robots[pair[0]].remaining_distance = ((robots[pair[0]].location[0] - tasks[pair[1]].location[0]) ** 2 + (robots[pair[0]].location[1] - tasks[pair[1]].location[1]) ** 2) ** 0.5
        robots[pair[0]].tasks_attempted = 1
        tasks[pair[1]].assigned_robot = robots[pair[0]]
    
    for time_step in range(max_time_steps):
#         print(f"\n--- Time Step {time_step + 1} ---")

        # Simulate time step
        unassigned_count, total_reward, total_success = simulate_time_step(robots, tasks, unassigned_robots, unassigned_tasks, suitability_method, 1.0, total_reward, total_success)
#         print(unassigned_count, len(robots), unassigned_robots, unassigned_tasks)
        # Periodically add new tasks and robots
        if add_tasks:
            task_max_id, total_tasks = add_new_tasks(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks)
        if add_robots:
            robot_max_id = add_new_robots(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add))

        # Periodically remove robots
        if remove_robots:
            remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove))

        # Reassign unassigned robots to unassigned tasks
        if len(unassigned_robots) > 0 and len(unassigned_tasks) > 0:
            unassigned_robots, unassigned_tasks, reassign_score, reassign_length = reassign_robots_to_tasks_with_method(robots, tasks, num_candidates, voting_method, suitability_method, unassigned_robots, unassigned_tasks, cbba_task_allocation)
            total_reassignment_time += reassign_length
            total_reassignment_score += reassign_score

    overall_success_rate = total_success / total_tasks
    print(f"CBBA: Total reward: {total_reward}, Overall success rate: {overall_success_rate:.2%}, Tasks completed: {total_success}, Reassignment Time: {total_reassignment_time}, Reassignment Score: {total_reassignment_score}")
#     for robot in robots:
#         print(f"Robot {robot.robot_id} attempted {robot.tasks_attempted} tasks and successfully completed {robot.tasks_successful} of them.")

    robots = copy.deepcopy(initial_robots)
    tasks = copy.deepcopy(initial_tasks)
    robot_max_id = len(robots)+1
    task_max_id = len(tasks)+1
    total_reward = 0.0
    total_success = 0.0
    total_tasks = num_tasks
    total_reassignment_time = 0.0
    total_reassignment_score = 0.0
    assigned_pairs = ssia_output[0]
    unassigned_robots = [robots[val].robot_id for val in ssia_output[1]]
    unassigned_tasks = [tasks[val].task_id for val in ssia_output[2]]
    
    for pair in assigned_pairs:
        robots[pair[0]].current_task = tasks[pair[1]]
        robots[pair[0]].remaining_distance = ((robots[pair[0]].location[0] - tasks[pair[1]].location[0]) ** 2 + (robots[pair[0]].location[1] - tasks[pair[1]].location[1]) ** 2) ** 0.5
        robots[pair[0]].tasks_attempted = 1
        tasks[pair[1]].assigned_robot = robots[pair[0]]
    
    for time_step in range(max_time_steps):
#         print(f"\n--- Time Step {time_step + 1} ---")

        # Simulate time step
        unassigned_count, total_reward, total_success = simulate_time_step(robots, tasks, unassigned_robots, unassigned_tasks, suitability_method, 1.0, total_reward, total_success)
#         print(unassigned_count, len(robots), unassigned_robots, unassigned_tasks)
        # Periodically add new tasks and robots
        if add_tasks:
            task_max_id, total_tasks = add_new_tasks(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks)
        if add_robots:
            robot_max_id = add_new_robots(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add))

        # Periodically remove robots
        if remove_robots:
            remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove))

        # Reassign unassigned robots to unassigned tasks
        if len(unassigned_robots) > 0 and len(unassigned_tasks) > 0:
            unassigned_robots, unassigned_tasks, reassign_score, reassign_length = reassign_robots_to_tasks_with_method(robots, tasks, num_candidates, voting_method, suitability_method, unassigned_robots, unassigned_tasks, ssia_task_allocation)
            total_reassignment_time += reassign_length
            total_reassignment_score += reassign_score

    overall_success_rate = total_success / total_tasks
    print(f"SSIA: Total reward: {total_reward}, Overall success rate: {overall_success_rate:.2%}, Tasks completed: {total_success}, Reassignment Time: {total_reassignment_time}, Reassignment Score: {total_reassignment_score}")
#     for robot in robots:
#         print(f"Robot {robot.robot_id} attempted {robot.tasks_attempted} tasks and successfully completed {robot.tasks_successful} of them.")

    robots = copy.deepcopy(initial_robots)
    tasks = copy.deepcopy(initial_tasks)
    robot_max_id = len(robots)+1
    task_max_id = len(tasks)+1
    total_reward = 0.0
    total_success = 0.0
    total_tasks = num_tasks
    total_reassignment_time = 0.0
    total_reassignment_score = 0.0
    assigned_pairs = ilp_output[0]
    unassigned_robots = [robots[val].robot_id for val in ilp_output[1]]
    unassigned_tasks = [tasks[val].task_id for val in ilp_output[2]]
    
    for pair in assigned_pairs:
        robots[pair[0]].current_task = tasks[pair[1]]
        robots[pair[0]].remaining_distance = ((robots[pair[0]].location[0] - tasks[pair[1]].location[0]) ** 2 + (robots[pair[0]].location[1] - tasks[pair[1]].location[1]) ** 2) ** 0.5
        robots[pair[0]].tasks_attempted = 1
        tasks[pair[1]].assigned_robot = robots[pair[0]]
    
    for time_step in range(max_time_steps):
#         print(f"\n--- Time Step {time_step + 1} ---")

        # Simulate time step
        unassigned_count, total_reward, total_success = simulate_time_step(robots, tasks, unassigned_robots, unassigned_tasks, suitability_method, 1.0, total_reward, total_success)
#         print(unassigned_count, len(robots), unassigned_robots, unassigned_tasks)
        # Periodically add new tasks and robots
        if add_tasks:
            task_max_id, total_tasks = add_new_tasks(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks)
        if add_robots:
            robot_max_id = add_new_robots(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add))

        # Periodically remove robots
        if remove_robots:
            remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove))

        # Reassign unassigned robots to unassigned tasks
        if len(unassigned_robots) > 0 and len(unassigned_tasks) > 0:
            unassigned_robots, unassigned_tasks, reassign_score, reassign_length = reassign_robots_to_tasks_with_method(robots, tasks, num_candidates, voting_method, suitability_method, unassigned_robots, unassigned_tasks, ilp_task_allocation)
            total_reassignment_time += reassign_length
            total_reassignment_score += reassign_score

    overall_success_rate = total_success / total_tasks
    print(f"ILP: Total reward: {total_reward}, Overall success rate: {overall_success_rate:.2%}, Tasks completed: {total_success}, Reassignment Time: {total_reassignment_time}, Reassignment Score: {total_reassignment_score}")
#     for robot in robots:
#         print(f"Robot {robot.robot_id} attempted {robot.tasks_attempted} tasks and successfully completed {robot.tasks_successful} of them.")


if __name__ == "__main__":
#     voting_methods = ["rank_assignments_borda", "rank_assignments_approval", "rank_assignments_majority_judgment", "rank_assignments_cumulative_voting", "rank_assignments_condorcet_method", "rank_assignments_range"]
    voting_methods = ["rank_assignments_range"]
#     suitability_methods = ["evaluate_suitability_loose", "evaluate_suitability_strict", "evaluate_suitability_distance", "evaluate_suitability_priority"]
    suitability_methods = ["evaluate_suitability_loose","evaluate_suitability_distance"]
    max_time_steps = 1000
    add_tasks = True
    add_robots = True
    remove_robots = True
    # param_combinations = []
    for i in [500]:
        for j in [500]:
            for nc in [5000]:
                for vm in voting_methods:
                    for sm in suitability_methods:
                        for k in range(0,10):
                            # param_combinations.append((i, j, nc, vm, sm, max_time_steps, add_tasks, add_robots, remove_robots, 10, 10, 10))
    # with multiprocessing.Pool() as pool:
        # pool.starmap(main_simulation, param_combinations)
                            main_simulation(i, j, nc, vm, sm, max_time_steps, add_tasks, add_robots, remove_robots,10,10,10)

500 500 5000 rank_assignments_range evaluate_suitability_loose 5414.490614793983 369227.3 7072.490614793985 8843820.6 6938.490614793982 16206.6 7353.490614793983 6411574.9
Voting: Total reward: 27511.024114946373, Overall success rate: 90.99%, Tasks completed: 4951.0, Reassignment Time: 6376478.099999983, Reassignment Score: 119860.79778484136
CBBA: Total reward: 27938.405187276083, Overall success rate: 92.83%, Tasks completed: 5040.0, Reassignment Time: 19152968.800000016, Reassignment Score: 123056.80896249066
SSIA: Total reward: 27203.892708396772, Overall success rate: 90.17%, Tasks completed: 4979.0, Reassignment Time: 113911.10000000021, Reassignment Score: 121800.09584112182
ILP: Total reward: 28303.886637219974, Overall success rate: 92.91%, Tasks completed: 5192.0, Reassignment Time: 65298520.79999998, Reassignment Score: 121265.53186074467
500 500 5000 rank_assignments_range evaluate_suitability_loose 5440.325556173581 371846.0 7035.825556173575 9071666.3 6897.325556173574 1

C:\Users\owner\AppData\Local\Temp\ipykernel_22056\2839024595.py:571: RuntimeWarning: divide by zero encountered in scalar divide
  score = score / distance_to_task


Voting: Total reward: 28099.954798173985, Overall success rate: 92.87%, Tasks completed: 5070.0, Reassignment Time: 6685406.499999995, Reassignment Score: 29177.40728977383
CBBA: Total reward: 28410.863947380534, Overall success rate: 91.22%, Tasks completed: 5067.0, Reassignment Time: 2224924.5999999987, Reassignment Score: 41296.02016677217
SSIA: Total reward: 25228.88485221753, Overall success rate: 84.10%, Tasks completed: 4576.0, Reassignment Time: 62596.09999999995, Reassignment Score: 25420.152468576256
ILP: Total reward: 27258.27266820318, Overall success rate: 92.96%, Tasks completed: 4993.0, Reassignment Time: 36127897.599999994, Reassignment Score: 46993.868856867455
500 500 5000 rank_assignments_range evaluate_suitability_distance 660.0052831727602 380099.1 6900.605703405866 2648459.9 5168.2743222078325 15902.4 6965.558574120986 6418265.6
Voting: Total reward: 19781.15058705119, Overall success rate: 64.51%, Tasks completed: 3577.0, Reassignment Time: 10036697.60000001, Rea

KeyboardInterrupt: 

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "bigscience/bloomz-560m"

# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForCausalLM.from_pretrained(checkpoint)

# inputs = tokenizer.encode(prompt[0], return_tensors="pt")
# outputs = model.generate(inputs, max_new_tokens=600)
# print(tokenizer.decode(outputs[0]))

In [18]:
import huggingface_hub
huggingface_hub.login('***REMOVED***')

In [19]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.98s/it]


In [20]:
model = model.to("cuda")

# Function to generate text
def generate_text(prompt, max_length=1000):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(inputs.input_ids, max_length=max_length, do_sample=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test the model
prompt = "Once upon a time in a land far, far away,"
print(generate_text(prompt))


Once upon a time in a land far, far away, there was a magical kingdom where all the creatures lived in harmony. But one day, a dark force threatened to destroy their peaceful way of life. The creatures banded together to defend their home, and in the process, they discovered the true meaning of courage and friendship.

In this kingdom, there lived a wise old owl named Hootie who was known for his wisdom and insight. Hootie had been watching the creatures of the kingdom for many years and had seen the dark force growing stronger by the day. He knew that something had to be done to stop it, but he didn't know what.

One day, Hootie called a meeting of all the creatures of the kingdom to discuss the problem. The creatures gathered around him, their eyes filled with fear and uncertainty. Hootie looked at them and said, "My dear friends, we are in grave danger. The dark force that threatens our kingdom is growing stronger every day. We must find a way to stop it before it's too late."

The 

In [21]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import MultiComparison
from statsmodels.stats.anova import anova_lm

# Load the data
file_path = 'VBTAData.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Define categorical and continuous variables
categorical_factors = ['Voting_Method', 'Suitability_Method']  # Replace with actual categorical column names
continuous_factors = ['Robots', 'Tasks']   # Replace with actual continuous column names
# dependent_var = 'Score_Voting'                   # Replace with your dependent variable name
dependent_var = 'Time_Voting'                   # Replace with your dependent variable name

# Convert categorical variables to category dtype
for factor in categorical_factors:
    data[factor] = data[factor].astype('category')

# Define the formula for the ANCOVA model
formula = f"{dependent_var} ~ {' * '.join(categorical_factors)} + {' + '.join(continuous_factors)}"

# Fit the ANCOVA model
model = ols(formula, data=data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)  # Type 2 ANOVA/ANCOVA

# Print ANCOVA results
print("Two-way ANCOVA results:")
print(anova_table)

# Post hoc analysis: Pairwise comparisons on adjusted means
print("\nPost hoc analysis (pairwise comparisons):")
for factor in categorical_factors:
    print(f"\nComparisons within {factor} (adjusted for covariates):")
    
    # Pairwise comparisons using t-tests
    mc = MultiComparison(data[dependent_var], data[factor])
    tukey_result = mc.tukeyhsd()
    
    # Display Tukey's test results for adjusted means
    print(tukey_result)

Two-way ANCOVA results:
                                        sum_sq       df            F  \
Voting_Method                     3.174761e+15      4.0   344.271446   
Suitability_Method                3.633505e+12      3.0     0.525357   
Voting_Method:Suitability_Method  5.143658e+12     12.0     0.185926   
Robots                            1.054736e+16      1.0  4575.028956   
Tasks                             1.140436e+16      1.0  4946.761087   
Residual                          4.513551e+16  19578.0          NaN   

                                         PR(>F)  
Voting_Method                     6.711993e-287  
Suitability_Method                 6.648329e-01  
Voting_Method:Suitability_Method   9.989597e-01  
Robots                             0.000000e+00  
Tasks                              0.000000e+00  
Residual                                    NaN  

Post hoc analysis (pairwise comparisons):

Comparisons within Voting_Method (adjusted for covariates):
                 

In [22]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Load the data
file_path = 'VBTAData.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Define column names
# group_columns = ['Score_Voting', 'Score_CBBA', 'Score_SSIA', 'Score_ILP']  # Replace with your actual column names for each group
group_columns = ['Time_Voting', 'Time_CBBA', 'Time_SSIA', 'Time_ILP']  # Replace with your actual column names for each group
covariates = ['Robots', 'Tasks']  # Replace with actual covariate column names

data = data[data['Voting_Method'] == 'rank_assignments_range']

data = data[data['Suitability_Method'] == 'evaluate_suitability_loose']

# Melt the data from wide to long format
data_long = data.melt(id_vars=covariates, value_vars=group_columns, var_name='group', value_name='dependent')

# Convert the group variable to categorical
data_long['group'] = data_long['group'].astype('category')

# Define the formula for the ANCOVA model
formula = f"dependent ~ group + {' + '.join(covariates)}"

# Fit the ANCOVA model
model = ols(formula, data=data_long).fit()
anova_table = sm.stats.anova_lm(model, typ=2)  # Type 2 ANCOVA for better control over interactions

# Print ANCOVA results
print("ANCOVA results:")
print(anova_table)

# Check if the group factor is significant
if anova_table.loc['group', 'PR(>F)'] < 0.05:
    print("\nThe group factor is significant. Proceeding with post hoc tests.")
    
    # Post hoc analysis using Tukey's HSD
    print("\nPost hoc analysis (Tukey's HSD on adjusted means):")
    tukey_result = pairwise_tukeyhsd(data_long['dependent'], data_long['group'], alpha=0.05)
    print(tukey_result)
else:
    print("\nThe group factor is not significant. No post hoc test needed.")


ANCOVA results:
                sum_sq      df           F         PR(>F)
group     5.865748e+14     3.0  233.766498  1.620202e-139
Robots    7.722562e+14     1.0  923.297264  2.858098e-182
Tasks     5.819917e+14     1.0  695.820039  2.786051e-141
Residual  3.273714e+15  3914.0         NaN            NaN

The group factor is significant. Proceeding with post hoc tests.

Post hoc analysis (Tukey's HSD on adjusted means):
            Multiple Comparison of Means - Tukey HSD, FWER=0.05            
  group1     group2     meandiff   p-adj      lower        upper     reject
---------------------------------------------------------------------------
Time_CBBA    Time_ILP  476152.4754    0.0   349931.5305  602373.4203   True
Time_CBBA   Time_SSIA  -477618.259    0.0  -603839.2039 -351397.3141   True
Time_CBBA Time_Voting -438219.1265    0.0  -564440.0714 -311998.1816   True
 Time_ILP   Time_SSIA -953770.7344    0.0 -1079991.6793 -827549.7895   True
 Time_ILP Time_Voting -914371.6019    0.0 -1

In [ ]:
# find a way to use LLM to calculate the similarity score values using a semantic understanding of the text ("Task" and "Robot" descriptions) and the suitability matrix
# import multiprocessing

def main_simulation(num_robots: int, num_tasks: int, num_candidates: int, voting_method: str, suitability_method: str, max_time_steps: int, add_tasks: bool, add_robots: bool, remove_robots: bool, tasks_to_add: int = 1, robots_to_add: int = 1, robots_to_remove: int = 1):
    robots = [generate_random_robot_profile(f"R{i+1}") for i in range(num_robots)]
    tasks = [generate_random_task_description(f"T{i+1}") for i in range(num_tasks)]
    initial_robots = copy.deepcopy(robots)
    initial_tasks = copy.deepcopy(tasks)
    robot_max_id = len(robots)+1
    task_max_id = len(tasks)+1
    total_reward = 0.0
    total_success = 0.0
    total_tasks = num_tasks
    total_reassignment_time = 0.0
    total_reassignment_score = 0.0

    # Calculate suitability matrix
    suitability_matrix = calculate_suitability_matrix(robots, tasks, suitability_method)
    
    # Perform task assignment using voting
    output, score, length = assign_tasks_with_voting(robots, tasks, suitability_matrix, num_candidates, voting_method)
        
    assigned_pairs = output[0]
    unassigned_robots = [robots[val].robot_id for val in output[1]]
    unassigned_tasks = [tasks[val].task_id for val in output[2]]
#     print(unassigned_robots, unassigned_tasks, task_max_id)
#     print(num_robots, num_tasks, num_candidates, voting_method, suitability_method, score, length)
#     print(assigned_pairs)

    cbba_output, cbba_score, cbba_length = assign_tasks_with_method(cbba_task_allocation,suitability_matrix)
    ssia_output, ssia_score, ssia_length = assign_tasks_with_method(ssia_task_allocation,suitability_matrix)
    ilp_output, ilp_score, ilp_length = assign_tasks_with_method(ilp_task_allocation,suitability_matrix)
    
    print(num_robots, num_tasks, num_candidates, voting_method, suitability_method, score, length, cbba_score, cbba_length, ssia_score, ssia_length, ilp_score, ilp_length)

    for pair in assigned_pairs:
        robots[pair[0]].current_task = tasks[pair[1]]
        robots[pair[0]].remaining_distance = ((robots[pair[0]].location[0] - tasks[pair[1]].location[0]) ** 2 + (robots[pair[0]].location[1] - tasks[pair[1]].location[1]) ** 2) ** 0.5
        robots[pair[0]].tasks_attempted = 1
        tasks[pair[1]].assigned_robot = robots[pair[0]]
    
    for time_step in range(max_time_steps):
#         print(f"\n--- Time Step {time_step + 1} ---")

        # Simulate time step
        unassigned_count, total_reward, total_success = simulate_time_step(robots, tasks, unassigned_robots, unassigned_tasks, suitability_method, 1.0, total_reward, total_success)
#         print(unassigned_count, len(robots), unassigned_robots, unassigned_tasks)
        # Periodically add new tasks and robots
        if add_tasks:
            task_max_id, total_tasks = add_new_tasks(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks)
        if add_robots:
            robot_max_id = add_new_robots(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add))

        # Periodically remove robots
        if remove_robots:
            remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove))

        # Reassign unassigned robots to unassigned tasks
        if len(unassigned_robots) > 0 and len(unassigned_tasks) > 0:
            unassigned_robots, unassigned_tasks, reassign_score, reassign_length = reassign_robots_to_tasks(robots, tasks, num_candidates, voting_method, suitability_method, unassigned_robots, unassigned_tasks)
            total_reassignment_time += reassign_length
            total_reassignment_score += reassign_score

    overall_success_rate = total_success / total_tasks
    print(f"Voting: Total reward: {total_reward}, Overall success rate: {overall_success_rate:.2%}, Tasks completed: {total_success}, Reassignment Time: {total_reassignment_time}, Reassignment Score: {total_reassignment_score}")
#     for robot in robots:
#         print(f"Robot {robot.robot_id} attempted {robot.tasks_attempted} tasks and successfully completed {robot.tasks_successful} of them.")

    robots = copy.deepcopy(initial_robots)
    tasks = copy.deepcopy(initial_tasks)
    robot_max_id = len(robots)+1
    task_max_id = len(tasks)+1
    total_reward = 0.0
    total_success = 0.0
    total_tasks = num_tasks
    total_reassignment_time = 0.0
    total_reassignment_score = 0.0
    assigned_pairs = cbba_output[0]
    unassigned_robots = [robots[val].robot_id for val in cbba_output[1]]
    unassigned_tasks = [tasks[val].task_id for val in cbba_output[2]]
    
    for pair in assigned_pairs:
        robots[pair[0]].current_task = tasks[pair[1]]
        robots[pair[0]].remaining_distance = ((robots[pair[0]].location[0] - tasks[pair[1]].location[0]) ** 2 + (robots[pair[0]].location[1] - tasks[pair[1]].location[1]) ** 2) ** 0.5
        robots[pair[0]].tasks_attempted = 1
        tasks[pair[1]].assigned_robot = robots[pair[0]]
    
    for time_step in range(max_time_steps):
#         print(f"\n--- Time Step {time_step + 1} ---")

        # Simulate time step
        unassigned_count, total_reward, total_success = simulate_time_step(robots, tasks, unassigned_robots, unassigned_tasks, suitability_method, 1.0, total_reward, total_success)
#         print(unassigned_count, len(robots), unassigned_robots, unassigned_tasks)
        # Periodically add new tasks and robots
        if add_tasks:
            task_max_id, total_tasks = add_new_tasks(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks)
        if add_robots:
            robot_max_id = add_new_robots(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add))

        # Periodically remove robots
        if remove_robots:
            remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove))

        # Reassign unassigned robots to unassigned tasks
        if len(unassigned_robots) > 0 and len(unassigned_tasks) > 0:
            unassigned_robots, unassigned_tasks, reassign_score, reassign_length = reassign_robots_to_tasks_with_method(robots, tasks, num_candidates, voting_method, suitability_method, unassigned_robots, unassigned_tasks, cbba_task_allocation)
            total_reassignment_time += reassign_length
            total_reassignment_score += reassign_score

    overall_success_rate = total_success / total_tasks
    print(f"CBBA: Total reward: {total_reward}, Overall success rate: {overall_success_rate:.2%}, Tasks completed: {total_success}, Reassignment Time: {total_reassignment_time}, Reassignment Score: {total_reassignment_score}")
#     for robot in robots:
#         print(f"Robot {robot.robot_id} attempted {robot.tasks_attempted} tasks and successfully completed {robot.tasks_successful} of them.")

    robots = copy.deepcopy(initial_robots)
    tasks = copy.deepcopy(initial_tasks)
    robot_max_id = len(robots)+1
    task_max_id = len(tasks)+1
    total_reward = 0.0
    total_success = 0.0
    total_tasks = num_tasks
    total_reassignment_time = 0.0
    total_reassignment_score = 0.0
    assigned_pairs = ssia_output[0]
    unassigned_robots = [robots[val].robot_id for val in ssia_output[1]]
    unassigned_tasks = [tasks[val].task_id for val in ssia_output[2]]
    
    for pair in assigned_pairs:
        robots[pair[0]].current_task = tasks[pair[1]]
        robots[pair[0]].remaining_distance = ((robots[pair[0]].location[0] - tasks[pair[1]].location[0]) ** 2 + (robots[pair[0]].location[1] - tasks[pair[1]].location[1]) ** 2) ** 0.5
        robots[pair[0]].tasks_attempted = 1
        tasks[pair[1]].assigned_robot = robots[pair[0]]
    
    for time_step in range(max_time_steps):
#         print(f"\n--- Time Step {time_step + 1} ---")

        # Simulate time step
        unassigned_count, total_reward, total_success = simulate_time_step(robots, tasks, unassigned_robots, unassigned_tasks, suitability_method, 1.0, total_reward, total_success)
#         print(unassigned_count, len(robots), unassigned_robots, unassigned_tasks)
        # Periodically add new tasks and robots
        if add_tasks:
            task_max_id, total_tasks = add_new_tasks(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks)
        if add_robots:
            robot_max_id = add_new_robots(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add))

        # Periodically remove robots
        if remove_robots:
            remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove))

        # Reassign unassigned robots to unassigned tasks
        if len(unassigned_robots) > 0 and len(unassigned_tasks) > 0:
            unassigned_robots, unassigned_tasks, reassign_score, reassign_length = reassign_robots_to_tasks_with_method(robots, tasks, num_candidates, voting_method, suitability_method, unassigned_robots, unassigned_tasks, ssia_task_allocation)
            total_reassignment_time += reassign_length
            total_reassignment_score += reassign_score

    overall_success_rate = total_success / total_tasks
    print(f"SSIA: Total reward: {total_reward}, Overall success rate: {overall_success_rate:.2%}, Tasks completed: {total_success}, Reassignment Time: {total_reassignment_time}, Reassignment Score: {total_reassignment_score}")
#     for robot in robots:
#         print(f"Robot {robot.robot_id} attempted {robot.tasks_attempted} tasks and successfully completed {robot.tasks_successful} of them.")

    robots = copy.deepcopy(initial_robots)
    tasks = copy.deepcopy(initial_tasks)
    robot_max_id = len(robots)+1
    task_max_id = len(tasks)+1
    total_reward = 0.0
    total_success = 0.0
    total_tasks = num_tasks
    total_reassignment_time = 0.0
    total_reassignment_score = 0.0
    assigned_pairs = ilp_output[0]
    unassigned_robots = [robots[val].robot_id for val in ilp_output[1]]
    unassigned_tasks = [tasks[val].task_id for val in ilp_output[2]]
    
    for pair in assigned_pairs:
        robots[pair[0]].current_task = tasks[pair[1]]
        robots[pair[0]].remaining_distance = ((robots[pair[0]].location[0] - tasks[pair[1]].location[0]) ** 2 + (robots[pair[0]].location[1] - tasks[pair[1]].location[1]) ** 2) ** 0.5
        robots[pair[0]].tasks_attempted = 1
        tasks[pair[1]].assigned_robot = robots[pair[0]]
    
    for time_step in range(max_time_steps):
#         print(f"\n--- Time Step {time_step + 1} ---")

        # Simulate time step
        unassigned_count, total_reward, total_success = simulate_time_step(robots, tasks, unassigned_robots, unassigned_tasks, suitability_method, 1.0, total_reward, total_success)
#         print(unassigned_count, len(robots), unassigned_robots, unassigned_tasks)
        # Periodically add new tasks and robots
        if add_tasks:
            task_max_id, total_tasks = add_new_tasks(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks)
        if add_robots:
            robot_max_id = add_new_robots(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add))

        # Periodically remove robots
        if remove_robots:
            remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove))

        # Reassign unassigned robots to unassigned tasks
        if len(unassigned_robots) > 0 and len(unassigned_tasks) > 0:
            unassigned_robots, unassigned_tasks, reassign_score, reassign_length = reassign_robots_to_tasks_with_method(robots, tasks, num_candidates, voting_method, suitability_method, unassigned_robots, unassigned_tasks, ilp_task_allocation)
            total_reassignment_time += reassign_length
            total_reassignment_score += reassign_score

    overall_success_rate = total_success / total_tasks
    print(f"ILP: Total reward: {total_reward}, Overall success rate: {overall_success_rate:.2%}, Tasks completed: {total_success}, Reassignment Time: {total_reassignment_time}, Reassignment Score: {total_reassignment_score}")
#     for robot in robots:
#         print(f"Robot {robot.robot_id} attempted {robot.tasks_attempted} tasks and successfully completed {robot.tasks_successful} of them.")

    # TODO: Implement the LLM-based reassignment method
    robots = copy.deepcopy(initial_robots)
    tasks = copy.deepcopy(initial_tasks)
    robot_max_id = len(robots)+1
    task_max_id = len(tasks)+1
    total_reward = 0.0
    total_success = 0.0
    total_tasks = num_tasks
    total_reassignment_time = 0.0
    total_reassignment_score = 0.0
    assigned_pairs = ilp_output[0]
    unassigned_robots = [robots[val].robot_id for val in ilp_output[1]]
    unassigned_tasks = [tasks[val].task_id for val in ilp_output[2]]
    
    for pair in assigned_pairs:
        robots[pair[0]].current_task = tasks[pair[1]]
        robots[pair[0]].remaining_distance = ((robots[pair[0]].location[0] - tasks[pair[1]].location[0]) ** 2 + (robots[pair[0]].location[1] - tasks[pair[1]].location[1]) ** 2) ** 0.5
        robots[pair[0]].tasks_attempted = 1
        tasks[pair[1]].assigned_robot = robots[pair[0]]

        for _ in range(max_time_steps):
            # Simulate time step
            # use LLM to calculate the similarity score values using a semantic understanding of the text ("Task" and "Robot" descriptions) and the suitability matrix
            
            # Periodically add new tasks and robots
            if add_tasks:
                task_max_id, total_tasks = add_new_tasks(tasks, unassigned_tasks, task_max_id, random.randint(0, tasks_to_add), total_tasks)
            if add_robots:
                robot_max_id = add_new_robots(robots, unassigned_robots, robot_max_id, random.randint(0, robots_to_add))
            # Periodically remove robots
            if remove_robots:
                remove_random_robots(robots, tasks, unassigned_robots, unassigned_tasks, random.randint(0, robots_to_remove))
            # Reassign unassigned robots to unassigned tasks
            # use LLM to reassign robots to tasks
            


if __name__ == "__main__":
#     voting_methods = ["rank_assignments_borda", "rank_assignments_approval", "rank_assignments_majority_judgment", "rank_assignments_cumulative_voting", "rank_assignments_condorcet_method", "rank_assignments_range"]
    voting_methods = ["rank_assignments_range"]
#     suitability_methods = ["evaluate_suitability_loose", "evaluate_suitability_strict", "evaluate_suitability_distance", "evaluate_suitability_priority"]
    suitability_methods = ["evaluate_suitability_loose","evaluate_suitability_distance"]
    max_time_steps = 1000
    add_tasks = True
    add_robots = True
    remove_robots = True
    # param_combinations = []
    for i in [500]:
        for j in [500]:
            for nc in [5000]:
                for vm in voting_methods:
                    for sm in suitability_methods:
                        for k in range(0,10):
                            # param_combinations.append((i, j, nc, vm, sm, max_time_steps, add_tasks, add_robots, remove_robots, 10, 10, 10))
    # with multiprocessing.Pool() as pool:
        # pool.starmap(main_simulation, param_combinations)
                            main_simulation(i, j, nc, vm, sm, max_time_steps, add_tasks, add_robots, remove_robots,10,10,10)